In [ ]:
# ==============================================================================
# MÓDULO CORE
# Contém constantes, classes de dados e funções utilitárias compartilhadas
# por toda a aplicação.
# ==============================================================================

# ----------------------------------------
# Importações de Bibliotecas Padrão
# ----------------------------------------
import os
import base64
import copy
import datetime
from pathlib import Path
from uuid import uuid4
from io import BytesIO
from dataclasses import dataclass, field
from typing import List, Dict, Any
from collections import defaultdict

# ----------------------------------------
# Importações de Bibliotecas de Terceiros
# ----------------------------------------
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Funções do SciPy
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit, minimize_scalar
from scipy.stats import linregress
# Exige scipy>=1.6 para a localização de cumtrapz em .integrate
from scipy.integrate import cumtrapz

# ==============================================================================
# Constantes Globais
# ==============================================================================
R = 8.314  # Constante universal dos gases (J/mol.K)

# ==============================================================================
# Estruturas de Dados Centrais
# ==============================================================================

@dataclass
class SinteringDataRecord:
    """
    Uma estrutura padronizada para armazenar um conjunto de dados de sinterização,
    seus metadados e parâmetros associados. Facilita a passagem de dados
    entre os diferentes módulos da aplicação.
    """
    ensaio_id: int
    Ea: float
    tipo_dado_y: str
    df: pd.DataFrame
    metadata: dict = field(default_factory=dict)

class DataHistory:
    """
    Classe para armazenar o histórico de estados de um DataFrame, permitindo
    operações de "undo" (desfazer) ao longo do processamento.
    """
    def __init__(self):
        self.history: List[Dict[str, Any]] = []

    def push(self, data: pd.DataFrame, module_name: str):
        """Adiciona uma cópia do estado atual dos dados ao histórico."""
        record = {
            'timestamp': datetime.datetime.now(),
            'module': module_name,
            'columns': list(data.columns),
            'data': copy.deepcopy(data)
        }
        self.history.append(record)

    def pop(self) -> Optional[Dict[str, Any]]:
        """Remove e retorna o último estado salvo no histórico."""
        return self.history.pop() if self.history else None

    def peek(self) -> Optional[Dict[str, Any]]:
        """Visualiza o último estado salvo sem removê-lo."""
        return self.history[-1] if self.history else None

    def get_all(self) -> List[Dict[str, Any]]:
        """Retorna uma cópia de todo o histórico."""
        return list(self.history)

# ==============================================================================
# Funções Utilitárias da Interface
# ==============================================================================

def add_suffix_once(col: str, suffix: str) -> str:
    """Retorna a string 'col' com o 'suffix' adicionado, se ela ainda não o possuir."""
    return col if col.endswith(suffix) else f"{col}{suffix}"

def criar_titulo(texto: str, nivel: int = 2) -> widgets.HTML:
    """Cria um título HTML para a interface, padronizando os cabeçalhos de seção."""
    return widgets.HTML(f"<h{nivel}>{texto}</h{nivel}>")

def exibir_mensagem(msg: str):
    """Exibe uma mensagem informativa na interface com formatação em azul."""
    display(HTML(f"<p style='color:blue; font-style:italic;'>{msg}</p>"))

def exibir_erro(msg: str):
    """Exibe uma mensagem de erro na interface com destaque em vermelho e negrito."""
    display(HTML(f"<p style='color:red; font-weight:bold;'>ERRO: {msg}</p>"))

def criar_caixa_colapsavel(titulo: str, conteudo: widgets.Widget, aberto: bool = False) -> widgets.Accordion:
    """Cria uma caixa colapsável (Accordion) para organizar seções da interface."""
    acc = widgets.Accordion(children=[conteudo])
    acc.set_title(0, titulo)
    if not aberto:
        acc.selected_index = None
    return acc

def gerar_link_download(df: pd.DataFrame, nome_arquivo: str = "dados.xlsx") -> HTML:
    """Gera um link HTML para download de um DataFrame como um arquivo Excel."""
    uid = uuid4().hex[:6]
    stem = Path(nome_arquivo).stem
    final_name = f"{stem}_{uid}.xlsx"

    # Usa um buffer na memória em vez de salvar em /tmp
    output = BytesIO()
    with pd.ExcelWriter(output, engine="openpyxl") as writer:
        df.to_excel(writer, index=False)

    # Codifica o buffer em base64 para embutir no link
    b64 = base64.b64encode(output.getvalue()).decode()

    return HTML(f'<a download="{final_name}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" target="_blank">Clique aqui para baixar: {final_name}</a>')

# ==============================================================================
# Modelos Matemáticos
# ==============================================================================

def boltzmann_sigmoid(x, A1, A2, x0, dx):
    """Função sigmoidal de Boltzmann (4 parâmetros)."""
    exp_term = np.exp(np.clip((x - x0) / dx, -700, 700))
    return A2 + (A1 - A2) / (1 + exp_term)

def generalized_logistic_stable(x, A1, A2, x0, b, c):
    """Função Logística Generalizada (5 parâmetros), numericamente estável."""
    z = -(x - x0) / b
    # Evita overflow em np.exp(z) quando z é grande
    log_1_plus_exp_z = np.where(z > 30, z, np.log1p(np.exp(z)))
    denominator = np.exp(c * log_1_plus_exp_z)
    return A2 + (A1 - A2) / (denominator + 1e-12) # Evita divisão por zero

ImportError: cannot import name 'cumtrapz' from 'scipy.integrate' (/usr/local/lib/python3.11/dist-packages/scipy/integrate/__init__.py)

In [ ]:
# modulo1_interface.py

import numpy as np
import ipywidgets as widgets
import core

class EaSelectionWidget:
    """Widget reutilizável para selecionar Energias de Ativação (Ea)."""
    def __init__(self):
        self._build_ui()

    def _build_ui(self):
        self.radio_energy_choice = widgets.RadioButtons(
            options=[("Discretas", "discretas"), ("Intervalo", "intervalo")],
            value="discretas",
            description="Modo de Seleção de Ea:",
            style={'description_width': 'initial'}
        )
        self.dropdown_num_energies = widgets.Dropdown(
            options=[(f"{i} energias", i) for i in range(1, 11)],
            value=3,
            description="Quantidade:"
        )
        self.btn_energy_add = widgets.Button(description="Gerar Campos", button_style='info')
        self.energies_box = widgets.VBox([])
        self.float_min = widgets.FloatText(value=100.0, description="Ea Mín (kJ/mol):")
        self.float_max = widgets.FloatText(value=2000.0, description="Ea Máx (kJ/mol):")
        self.num_points = widgets.IntText(value=20, description="Nº de Pontos:")

        self.discretas_container = widgets.VBox([
            self.dropdown_num_energies,
            self.btn_energy_add,
            self.energies_box
        ])
        self.intervalo_container = widgets.VBox(
            [self.float_min, self.float_max, self.num_points],
            layout={'display': 'none'}
        )

        self.radio_energy_choice.observe(self._on_energy_choice_changed, names='value')
        self.btn_energy_add.on_click(self._on_add_energies)
        self._on_add_energies(None)

        self.ui = widgets.VBox([
            widgets.HTML("<b>Definir Energias de Ativação (Ea) para Análise</b>"),
            self.radio_energy_choice,
            self.discretas_container,
            self.intervalo_container
        ])

    def _on_energy_choice_changed(self, change):
        is_discrete = change['new'] == "discretas"
        self.discretas_container.layout.display = '' if is_discrete else 'none'
        self.intervalo_container.layout.display = 'none' if is_discrete else ''

    def _on_add_energies(self, _):
        n = self.dropdown_num_energies.value
        self.energies_box.children = [
            widgets.FloatText(
                value=round(100.0 + 50 * i, 1),
                description=f"Ea {i+1} (kJ/mol):"
            )
            for i in range(n)
        ]

    def get_ea_list(self) -> List[float]:
        if self.radio_energy_choice.value == "discretas":
            values = [ft.value for ft in self.energies_box.children if ft.value > 0]
            if not values:
                core.exibir_erro("Nenhuma energia válida informada no modo 'Discretas'.")
                return []
            return sorted(set(values))
        else:
            ea_min = self.float_min.value
            ea_max = self.float_max.value
            n_pts = self.num_points.value
            if ea_max <= ea_min or n_pts < 2:
                core.exibir_erro(
                    "No modo 'Intervalo', Ea Máx > Ea Mín e Nº de Pontos >= 2."
                )
                return []
            return list(np.linspace(ea_min, ea_max, n_pts))


In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from io import BytesIO

# Importando do seu arquivo core
from core import (
    DataHistory, criar_titulo, exibir_mensagem, exibir_erro, add_suffix_once
)

class Modulo2Importacao:
    """
    Módulo 2 – Importação de Dados.
    Gerencia o upload, mapeamento e conversão inicial dos dados experimentais.
    """
    def __init__(self):
        self.n_ensaios = 1
        self.dfs_ensaios = [None] * 4  # Suporta até 4 ensaios
        self.file_uploads = []
        self.history = DataHistory()
        self._build_ui()

    def _build_ui(self):
        self.title_label = criar_titulo("Módulo 2 – Importação e Reconhecimento de Dados", nivel=2)

        intro_text = """
        <div style="font-family: Arial, sans-serif; line-height: 1.5;">
            <p>Neste módulo, os dados experimentais são carregados a partir de arquivos .xlsx.
            É fundamental que cada arquivo contenha colunas para <b>Tempo</b>, <b>Temperatura</b> e um parâmetro de sinterização (<b>Densidade</b> ou <b>Retração</b>).</p>
        </div>
        """
        self.intro_widget = widgets.HTML(value=intro_text)

        self.dropdown_ensaios = widgets.Dropdown(
            options=[(f"{i} Teste{'s' if i > 1 else ''}", i) for i in range(1, 5)],
            value=1, description="Número de testes:"
        )
        self.btn_confirm_ensaios = widgets.Button(description="Confirmar Qtd.", button_style='success')
        self.btn_confirm_ensaios.on_click(self._confirmar_num_ensaios)

        self.upload_box = widgets.VBox([])
        self.out = widgets.Output()

        self.main_ui = widgets.VBox([
            self.title_label, self.intro_widget,
            widgets.HBox([self.dropdown_ensaios, self.btn_confirm_ensaios]),
            self.upload_box, self.out
        ])

        # Gera a UI para 1 ensaio por padrão na inicialização
        self._confirmar_num_ensaios(None)

    def _confirmar_num_ensaios(self, _=None):
        with self.out: clear_output()
        self.n_ensaios = self.dropdown_ensaios.value
        exibir_mensagem(f"{self.n_ensaios} teste(s) selecionado(s).")

        self.upload_box.children = []
        self.file_uploads = []

        for i in range(self.n_ensaios):
            uploader = widgets.FileUpload(accept='.xlsx', multiple=False, description="Carregar .xlsx")
            out_local = widgets.Output()
            map_box = widgets.VBox([])
            map_confirm_btn = widgets.Button(description="Confirmar Mapeamento", button_style='success', layout={'display': 'none'})

            conv_panel = widgets.VBox([
                widgets.HTML("<b>Parâmetros de Conversão (Retração -> Densidade)</b>"),
                widgets.FloatText(value=10.0, description="Tamanho Inicial (mm):"),
                widgets.FloatText(value=55.0, description="Densidade Inicial (%):"),
                widgets.FloatText(value=95.0, description="Densidade Final (%):")
            ])
            conv_params = {
                "tamanho_inicial": conv_panel.children[1],
                "dens_inicial": conv_panel.children[2],
                "dens_final": conv_panel.children[3]
            }
            btn_convert = widgets.Button(description="Converter para Densidade", button_style='warning')

            upload_info = {
                "uploader": uploader, "out_local": out_local, "map_box": map_box,
                "map_confirm_btn": map_confirm_btn, "conv_params": conv_params, "btn_convert": btn_convert
            }
            self.file_uploads.append(upload_info)

            uploader.observe(self._on_load_data(i), names='value')
            map_confirm_btn.on_click(self._on_confirm_mapping(i))
            btn_convert.on_click(self._on_convert(i))

            vbox_ensaio = widgets.VBox([
                widgets.HTML(f"<hr><b>Teste {i+1}</b>"), uploader, out_local,
                map_box, map_confirm_btn, conv_panel, btn_convert
            ])
            self.upload_box.children += (vbox_ensaio,)

    def _on_load_data(self, idx):
        def _cb(change):
            w = self.file_uploads[idx]
            with w["out_local"]:
                clear_output(wait=True)
                if not change.new: return
                try:
                    file_info = next(iter(change.new.values()))
                    content = file_info['content']
                    df = pd.read_excel(BytesIO(content))
                    self.dfs_ensaios[idx] = df
                    exibir_mensagem(f"Arquivo carregado com {len(df)} linhas. Colunas: {df.columns.tolist()}")
                    self._create_mapping_ui_for_ensaio(idx, df)
                except Exception as e:
                    exibir_erro(f"Erro ao ler arquivo: {e}")
        return _cb

    def _create_mapping_ui_for_ensaio(self, idx, df):
        w = self.file_uploads[idx]
        options = [
            ('Ignorar', 'ignore'), ('Tempo (s)', 'Time_s'), ('Tempo (min)', 'Time_min'),
            ('Tempo (h)', 'Time_h'), ('Temperatura (°C)', 'Temperature_C'),
            ('Retração Absoluta (mm)', 'RetracaoAbs'), ('Retração Percentual (%)', 'RetracaoPct'),
            ('Retração Relativa (ΔL/L₀)', 'RetracaoRel'), ('Densidade Percentual (%)', 'DensidadePct')
        ]
        w["map_box"].children = [widgets.Dropdown(options=options, value='ignore', description=col) for col in df.columns]
        w["map_confirm_btn"].layout.display = ''

    def _on_confirm_mapping(self, idx):
        def _cb(_):
            w = self.file_uploads[idx]
            with w["out_local"]:
                clear_output(wait=True)
                df = self.dfs_ensaios[idx]
                if df is None: exibir_erro("DataFrame não carregado."); return

                try:
                    rename_dict = {dd.description: dd.value for dd in w["map_box"].children if dd.value != 'ignore'}

                    # Reintroduz a validação de tempo ambíguo
                    time_mappings = [v for v in rename_dict.values() if v.startswith('Time_')]
                    if len(time_mappings) > 1:
                        raise ValueError(f"Mapeamento de tempo ambíguo. Selecione apenas um tipo de tempo. Encontrados: {time_mappings}")

                    df_temp = df[list(rename_dict.keys())].rename(columns=rename_dict).copy()

                    if 'Time_min' in df_temp.columns: df_temp['Time_s'] = df_temp.pop('Time_min') * 60
                    elif 'Time_h' in df_temp.columns: df_temp['Time_s'] = df_temp.pop('Time_h') * 3600

                    if not any(c.startswith('Time') for c in df_temp.columns) or \
                       not any(c.startswith('Temp') for c in df_temp.columns) or \
                       not any(c.startswith('Dens') or c.startswith('Retracao') for c in df_temp.columns):
                        raise ValueError("Mapeamento incompleto. Verifique se Tempo, Temperatura e Densidade/Retração foram mapeados.")

                    df_temp.columns = [add_suffix_once(c, f"_e{idx+1}") for c in df_temp.columns]
                    self.dfs_ensaios[idx] = df_temp
                    exibir_mensagem(f"Mapeamento salvo. Colunas finais: {df_temp.columns.tolist()}")
                    self.history.push(df_temp, f"M2:Mapeamento:{idx}")
                except Exception as e:
                    exibir_erro(f"{e}")
        return _cb

    def _on_convert(self, idx):
        def _cb(_):
            w = self.file_uploads[idx]
            with w["out_local"]:
                clear_output(wait=True)
                df = self.dfs_ensaios[idx]
                if df is None or df.empty: exibir_erro("Dados não mapeados."); return

                params = w["conv_params"]
                di, df_final, lo = params["dens_inicial"].value, params["dens_final"].value, params["tamanho_inicial"].value
                target_col = f"DensidadePct_e{idx+1}"

                if target_col in df.columns: exibir_mensagem("Dados já convertidos."); return

                df_temp = df.copy()
                try:
                    found_col, retracao_rel = None, None
                    prefixes = [('RetracaoRel', 1), ('RetracaoPct', 100.0), ('RetracaoAbs', lo)]
                    for prefix, divisor in prefixes:
                        col_name = f"{prefix}_e{idx+1}"
                        if col_name in df_temp:
                            if prefix == 'RetracaoAbs' and lo <= 0: raise ValueError("Tamanho inicial (lo) deve ser > 0.")
                            retracao_rel = df_temp[col_name] / divisor
                            found_col = col_name
                            break

                    if found_col is None: raise ValueError("Nenhuma coluna de retração encontrada para converter.")

                    df_temp[target_col] = di / (1 + retracao_rel)**3
                    min_calc, max_calc = df_temp[target_col].min(), df_temp[target_col].max()
                    if max_calc > min_calc:
                        df_temp[target_col] = di + (df_temp[target_col] - min_calc) * (df_final - di) / (max_calc - min_calc)

                    df_temp.drop(columns=[found_col], inplace=True)
                    self.dfs_ensaios[idx] = df_temp
                    exibir_mensagem(f"Conversão concluída. Coluna '{found_col}' substituída por '{target_col}'.")
                    self.history.push(df_temp, f"M2:Conversao:{idx}")
                except Exception as e:
                    exibir_erro(f"Erro na conversão: {e}")
        return _cb

    def validar_etapa(self) -> bool:
        """
        Verifica se todos os ensaios selecionados foram completamente processados.
        Retorna True se tudo estiver OK, False caso contrário.
        """
        with self.out:
            clear_output(wait=True)
        all_ok = True
        for i in range(self.n_ensaios):
            df = self.dfs_ensaios[i]
            if df is None or df.empty:
                exibir_erro(f"Teste {i+1}: Dados não foram carregados ou mapeados.")
                all_ok = False
                continue

            # Verifica se a conversão para densidade foi feita
            if not any(c.startswith(f"DensidadePct_e{i+1}") for c in df.columns):
                exibir_erro(f"Teste {i+1}: Dados não foram convertidos para densidade. Clique em 'Converter para Densidade'.")
                all_ok = False

        if all_ok:
            exibir_mensagem("Validação da Etapa 1 concluída com sucesso!")
        return all_ok

    def get_dfs(self):
        return self.dfs_ensaios[:self.n_ensaios]

In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d, PchipInterpolator, Akima1DInterpolator

# Importando do seu arquivo core
from core import (
    DataHistory, exibir_mensagem, exibir_erro, gerar_link_download
)

def orlandini_araujo_filter(df: pd.DataFrame, bin_size: int = 10) -> pd.DataFrame:
    """
    Aplica o filtro de Orlandini-Araújo, agregando dados em intervalos de tempo.
    """
    time_col = next((col for col in df.columns if col.startswith('Time_s')), None)
    temp_col = next((col for col in df.columns if col.startswith('Temperature_C')), None)
    dens_col = next((col for col in df.columns if col.startswith('DensidadePct')), None)

    if not (time_col and temp_col and dens_col):
        raise ValueError("Faltam colunas (Time_s, Temperature_C, DensidadePct) para Orlandini-Araújo.")

    dfc = df.copy()
    dfc['bin'] = np.floor(dfc[time_col] / bin_size).astype(int)

    # Preserva todas as colunas originais, pegando a média por bin
    agg_dict = {col: 'mean' for col in df.columns}
    grouped = dfc.groupby('bin').agg(agg_dict).reset_index(drop=True)
    return grouped

class Modulo3Recorte:
    """
    Módulo 3 - Recorte de Dados, Ajuste de Tempo, Visualização e Filtragem.
    """
    def __init__(self, dfs_ensaios: list):
        self.dfs_ensaios = [df.copy() for df in dfs_ensaios if df is not None and not df.empty]
        self.n_ensaios = len(self.dfs_ensaios)
        self.history = DataHistory()
        self._build_ui()

    def _build_ui(self):
        if self.n_ensaios == 0:
            self.main_ui = widgets.HTML("<b>Nenhum ensaio válido recebido do Módulo 2.</b>")
            return

        panels = [self._build_ensaio_panel(i) for i in range(self.n_ensaios)]
        self.accordion = widgets.Accordion(children=panels)
        for i in range(self.n_ensaios):
            self.accordion.set_title(i, f"Ensaio {i+1}")

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Módulo 3 - Recorte e Filtragem</h3>"),
            widgets.HTML("Ajuste o tempo, corte intervalos e aplique filtros para preparar os dados para análise."),
            self.accordion
        ])

    def _build_ensaio_panel(self, idx: int) -> widgets.HBox:
        log_out = widgets.Output()
        fig_out = widgets.Output(layout={'width': '50%'})

        # --- Controles ---
        radio_zerar = widgets.RadioButtons(options=[("Não", False), ("Sim", True)], description="Zerar Tempo?")
        btn_zerar = widgets.Button(description="Aplicar Ajuste", button_style='info')

        tmin, tmax = self._get_time_range(idx)
        slider_cut = widgets.FloatRangeSlider(value=(tmin, tmax), min=tmin, max=tmax, step=1.0, description='Corte (s):', continuous_update=False)
        btn_cut = widgets.Button(description="Aplicar Corte", button_style='info')

        filter_dd = widgets.Dropdown(
            options=[
                ('Nenhum', 'none'), ('Orlandini-Araujo', 'orlandini'),
                ('Savitzky-Golay', 'savgol'), ('Interpolação Linear', 'linear'),
                ('Interpolação Cúbica', 'cubic'), ('Interpolação PCHIP', 'pchip'),
                ('Interpolação Akima', 'akima')
            ], value='none', description='Filtro/Interp.:'
        )
        bin_widget = widgets.IntText(value=10, description="Bin (s):", layout={'display': 'none'})
        npoints_widget = widgets.IntText(value=200, description="Nº Pontos:", layout={'display': 'none'})
        btn_filter = widgets.Button(description="Aplicar Filtro", button_style='info')

        btn_plot = widgets.Button(description="Exibir Curvas", button_style='primary')
        btn_export = widgets.Button(description="Exportar Excel", button_style='primary')

        # --- Callbacks ---
        btn_zerar.on_click(lambda b: self._on_zerar_tempo(idx, radio_zerar.value, slider_cut, log_out))
        btn_cut.on_click(lambda b: self._on_corte_intervalo(idx, slider_cut.value, slider_cut, log_out))
        filter_dd.observe(lambda c: self._on_filter_change(c.new, bin_widget, npoints_widget), names='value')
        btn_filter.on_click(lambda b: self._on_aplicar_filtro(idx, filter_dd.value, bin_widget, npoints_widget, slider_cut, log_out))
        btn_plot.on_click(lambda b: self._on_plot(idx, fig_out, log_out))
        btn_export.on_click(lambda b: self._on_export(idx, log_out))

        # --- Layout ---
        controls = widgets.VBox([
            widgets.HTML("<b>Ajuste de Tempo</b>"), radio_zerar, btn_zerar,
            widgets.HTML("<hr><b>Corte de Intervalo</b>"), slider_cut, btn_cut,
            widgets.HTML("<hr><b>Filtragem / Interpolação</b>"), filter_dd, bin_widget, npoints_widget, btn_filter,
            widgets.HTML("<hr><b>Ações</b>"), widgets.HBox([btn_plot, btn_export]), log_out
        ], layout={'width': '50%'})

        return widgets.HBox([controls, fig_out])

    def _update_slider_range(self, idx, slider):
        tmin, tmax = self._get_time_range(idx)
        slider.min = tmin
        slider.max = tmax
        slider.value = (tmin, tmax)

    def _on_zerar_tempo(self, idx, should_zero, slider, out):
        with out:
            clear_output(wait=True)
            dfi = self.dfs_ensaios[idx]
            time_col = next((c for c in dfi.columns if c.startswith('Time_s')), None)
            if not time_col:
                exibir_erro("Coluna de tempo não encontrada."); return

            if should_zero:
                min_t = dfi[time_col].min()
                dfi[time_col] -= min_t
                dfi.sort_values(time_col, inplace=True)
                exibir_mensagem(f"Tempo zerado (subtraído {min_t:.2f} s).")
            else:
                exibir_mensagem("Tempo não alterado.")

            self.dfs_ensaios[idx] = dfi.reset_index(drop=True)
            self.history.push(self.dfs_ensaios[idx], f"M3:ZerarTempo:{idx}")
            self._update_slider_range(idx, slider)

    def _on_corte_intervalo(self, idx, time_range, slider, out):
        with out:
            clear_output(wait=True)
            dfi = self.dfs_ensaios[idx]
            time_col = next((c for c in dfi.columns if c.startswith('Time_s')), None)
            if not time_col:
                exibir_erro("Coluna de tempo não encontrada."); return

            t_min, t_max = time_range
            mask = (dfi[time_col] >= t_min) & (dfi[time_col] <= t_max)
            df_cut = dfi[mask].copy().reset_index(drop=True)

            exibir_mensagem(f"Corte aplicado. Mantidos {len(df_cut)} de {len(dfi)} pontos.")
            self.dfs_ensaios[idx] = df_cut
            self.history.push(df_cut, f"M3:Corte:{idx}")
            self._update_slider_range(idx, slider)

    def _on_filter_change(self, method, bin_w, npoints_w):
        bin_w.layout.display = '' if method == 'orlandini' else 'none'
        npoints_w.layout.display = '' if method in ('linear', 'cubic', 'pchip', 'akima') else 'none'

    def _on_aplicar_filtro(self, idx, method, bin_w, npoints_w, slider, out):
        with out:
            clear_output(wait=True)
            dfi = self.dfs_ensaios[idx]
            # Identificar colunas essenciais
            time_col = next((c for c in dfi.columns if c.startswith('Time_s')), None)
            temp_col = next((c for c in dfi.columns if c.startswith('Temperature_C')), None)
            dens_col = next((c for c in dfi.columns if c.startswith('DensidadePct')), None)

            if not (time_col and temp_col and dens_col):
                exibir_erro("Faltam colunas (Tempo, Temperatura, Densidade) para aplicar o filtro."); return

            try:
                if method == 'none':
                    exibir_mensagem("Nenhum filtro aplicado."); return

                elif method == 'orlandini':
                    df_filtered = orlandini_araujo_filter(dfi, bin_size=bin_w.value)
                    exibir_mensagem(f"Filtro Orlandini-Araujo aplicado (bin={bin_w.value}s).")

                elif method == 'savgol':
                    df_filtered = dfi.copy()
                    w = min(11, (len(dfi) // 2) * 2 + 1) # Janela adaptativa e ímpar
                    if w < 5: raise ValueError("Pontos insuficientes para Savitzky-Golay.")
                    df_filtered[dens_col] = savgol_filter(df_filtered[dens_col], w, 2)
                    df_filtered[temp_col] = savgol_filter(df_filtered[temp_col], w, 2) # Suaviza temp também
                    exibir_mensagem(f"Filtro Savitzky-Golay aplicado (janela={w}).")

                elif method in ('linear', 'cubic', 'pchip', 'akima'):
                    dfu = dfi.drop_duplicates(subset=[time_col]).sort_values(time_col)
                    if len(dfu) < 4 and method in ('cubic', 'pchip', 'akima'):
                         raise ValueError(f"Pontos insuficientes ({len(dfu)}) para interpolação '{method}'.")

                    n_points = npoints_w.value
                    new_time = np.linspace(dfu[time_col].min(), dfu[time_col].max(), n_points)

                    # Cria interpoladores para densidade e temperatura
                    if method == 'pchip':
                        f_dens = PchipInterpolator(dfu[time_col], dfu[dens_col])
                        f_temp = PchipInterpolator(dfu[time_col], dfu[temp_col])
                    elif method == 'akima':
                        f_dens = Akima1DInterpolator(dfu[time_col], dfu[dens_col])
                        f_temp = Akima1DInterpolator(dfu[time_col], dfu[temp_col])
                    else: # linear ou cubic
                        f_dens = interp1d(dfu[time_col], dfu[dens_col], kind=method, bounds_error=False, fill_value="extrapolate")
                        f_temp = interp1d(dfu[time_col], dfu[temp_col], kind=method, bounds_error=False, fill_value="extrapolate")

                    # Recria o DataFrame PRESERVANDO TODAS as colunas interpoladas
                    df_filtered = pd.DataFrame({
                        time_col: new_time,
                        dens_col: f_dens(new_time),
                        temp_col: f_temp(new_time)
                    })
                    exibir_mensagem(f"Interpolação '{method}' aplicada, gerando {n_points} pontos.")

                else:
                    exibir_erro("Método de filtro desconhecido."); return

                self.dfs_ensaios[idx] = df_filtered.reset_index(drop=True)
                self.history.push(df_filtered, f"M3:Filtro:{method}:{idx}")
                self._update_slider_range(idx, slider)

            except Exception as e:
                exibir_erro(f"Erro ao aplicar filtro: {e}")

    def _on_plot(self, idx, fig_out, log_out):
        with log_out: clear_output(wait=True)
        with fig_out: clear_output(wait=True)
        dfi = self.dfs_ensaios[idx]

        time_col = next((c for c in dfi.columns if c.startswith('Time_s')), None)
        temp_col = next((c for c in dfi.columns if c.startswith('Temperature_C')), None)
        dens_col = next((c for c in dfi.columns if c.startswith('DensidadePct')), None)

        if not (time_col and temp_col and dens_col):
            with log_out: exibir_erro("Faltam colunas essenciais para a plotagem."); return

        with fig_out:
            fig, axs = plt.subplots(1, 3, figsize=(15, 4), tight_layout=True)
            fig.suptitle(f"Ensaio {idx+1} - Visualização de Dados Atuais")
            axs[0].plot(dfi[time_col], dfi[dens_col], '.-')
            axs[0].set(xlabel="Tempo (s)", ylabel="Densidade (%)")
            axs[1].plot(dfi[temp_col], dfi[dens_col], 'r.-')
            axs[1].set(xlabel="Temperatura (°C)", ylabel="Densidade (%)")
            axs[2].plot(dfi[time_col], dfi[temp_col], 'g.-')
            axs[2].set(xlabel="Tempo (s)", ylabel="Temperatura (°C)")
            for ax in axs: ax.grid(True, alpha=0.5)
            plt.show()

    def _on_export(self, idx, log_out):
        with log_out:
            clear_output(wait=True)
            dfi = self.dfs_ensaios[idx]
            if dfi.empty:
                exibir_erro("Nada para exportar."); return
            link = gerar_link_download(dfi, nome_arquivo=f"Ensaio_{idx+1}_Filtrado.xlsx")
            display(link)

    def display(self):
        display(self.main_ui)

    def get_dfs(self):
        return self.dfs_ensaios

In [ ]:
# ==============================================================================
# MÓDULO 4 (CORRIGIDO): Roteador de Análise e Simulação Híbrida
# Nome do arquivo: modulo4_roteador.py
# ==============================================================================
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

# Importa as constantes e funções necessárias dos seus módulos
from core import criar_titulo, exibir_mensagem, exibir_erro, R

class Modulo4Roteador:
    """
    Este módulo atua como um roteador e implementa a abordagem híbrida.
    Ele agora se comunica com o controlador principal para navegar para a próxima etapa.
    """
    def __init__(self, dfs_recortados: list, controller):
        self.dfs_recortados = dfs_recortados
        self.controller = controller  # Referência ao MainInteractive
        self.df_final_para_analise = None
        self._build_ui()

    def _build_ui(self):
        title = criar_titulo("Módulo 4: Preparar Dados para Curva Mestra", 3)

        intro_text = widgets.HTML(
            "<p>Seus dados experimentais estão prontos. Escolha como deseja prosseguir:</p>"
            "<ul>"
            "<li><b>Caminho A:</b> Use se você já possui de 3 a 5 ensaios para a análise padrão.</li>"
            "<li><b>Caminho B:</b> Use se você possui apenas 1 ou 2 ensaios e deseja gerar dados sintéticos para enriquecer a análise.</li>"
            "</ul>"
        )

        self.path_choice_widget = widgets.RadioButtons(
            options=['A. Analisar o(s) ensaio(s) experimental(is)',
                     'B. Gerar dados sintéticos (Abordagem Híbrida)'],
            value='A. Analisar o(s) ensaio(s) experimental(is)',
            disabled=False
        )

        self.G_widget = widgets.FloatText(description="Tam. Grão (G) nm:", value=100.0)
        self.rho0_widget = widgets.FloatText(description="Dens. Verde (ρ₀ %):", value=55.0)
        self.num_sinteticos_widget = widgets.IntText(description="Nº Total de Ensaios:", value=5, min=2, max=10)

        self.hybrid_params_box = widgets.VBox([
            widgets.HTML("<b>Parâmetros para Simulação:</b>"),
            self.G_widget, self.rho0_widget, self.num_sinteticos_widget
        ])

        self.run_button = widgets.Button(description="Preparar Dados e Avançar", button_style='success', icon='cogs')
        self.run_button.on_click(self._on_run_button_clicked)

        self.output_placeholder = widgets.Output()

        def on_path_change(change):
            self.hybrid_params_box.layout.display = '' if 'B.' in change.new else 'none'
        self.path_choice_widget.observe(on_path_change, names='value')
        on_path_change({'new': self.path_choice_widget.value})

        self.main_ui = widgets.VBox([
            title, intro_text, self.path_choice_widget,
            self.hybrid_params_box, self.run_button, self.output_placeholder
        ])

    def display(self):
        display(self.main_ui)

    def _on_run_button_clicked(self, b):
        with self.output_placeholder:
            clear_output(wait=True)

            # Desabilita o botão para evitar cliques duplos
            b.disabled = True

            success = False
            if 'A.' in self.path_choice_widget.value:
                success = self._execute_caminho_a()
            elif 'B.' in self.path_choice_widget.value:
                success = self._execute_caminho_b()

            # Se o processamento foi bem-sucedido, avança para a próxima etapa
            if success and self.controller:
                exibir_mensagem("Dados preparados. Avançando para a próxima etapa...")
                # Chama a função do controlador para avançar
                self.controller.navigate_to_next_step()
            else:
                # Reabilita o botão se houve erro
                b.disabled = False


    def _execute_caminho_a(self) -> bool:
        exibir_mensagem("Caminho A selecionado. Preparando dados experimentais...")

        dfs_com_chave = []
        for i, df in enumerate(self.dfs_recortados):
            df_copy = df.copy()
            df_copy['Ensaio'] = i
            dfs_com_chave.append(df_copy)

        self.df_final_para_analise = pd.concat(dfs_com_chave, ignore_index=True)
        return True

    def _execute_caminho_b(self) -> bool:
        exibir_mensagem("Caminho B selecionado. Iniciando workflow híbrido...")

        if not self.dfs_recortados:
            exibir_erro("Nenhum dado experimental foi carregado e recortado.")
            return False

        df_calibracao = self.dfs_recortados[0]
        G_material = self.G_widget.value
        rho0_material_pct = self.rho0_widget.value
        total_ensaios = self.num_sinteticos_widget.value

        if not G_material > 0:
            exibir_erro("Por favor, insira um tamanho de grão válido (> 0).")
            return False

        exibir_mensagem("Etapa 1: Calibrando o modelo...")

        def modelo_gomez_hotza(X, A, Ea, n):
            tempo, temperatura_C = X
            temperatura_K = temperatura_C + 273.15
            R_kj = R / 1000.0
            rho0_fracao = rho0_material_pct / 100.0

            termo_integral = (A / G_material) * (R_kj * temperatura_K**2 / (Ea * taxa_aquecimento_K_s)) * np.exp(-Ea / (R_kj * temperatura_K))

            # Adiciona proteção para evitar overflow ou valores negativos dentro de **(1/n)
            termo_integral[termo_integral < 0] = 0

            porosidade_prevista = (1 - rho0_fracao) * np.exp(-n * termo_integral**(1/n))
            return (1 - porosidade_prevista) * 100.0 # Retorna em %

        time_col = next((c for c in df_calibracao.columns if c.startswith('Time_s')), None)
        temp_col = next((c for c in df_calibracao.columns if c.startswith('Temperature_C')), None)
        dens_col = next((c for c in df_calibracao.columns if c.startswith('DensidadePct')), None)

        t_real = df_calibracao[time_col].values
        T_real_C = df_calibracao[temp_col].values
        rho_real_pct = df_calibracao[dens_col].values

        taxa_aquecimento_K_s = np.mean(np.gradient(T_real_C, t_real))
        if taxa_aquecimento_K_s <= 0:
            exibir_erro("A taxa de aquecimento calculada é nula ou negativa. Verifique os dados de tempo e temperatura.")
            return False

        try:
            params_otimizados, _ = curve_fit(modelo_gomez_hotza, (t_real, T_real_C), rho_real_pct, p0=[1e10, 200, 0.5], maxfev=10000, bounds=([0, 10, 0.1], [np.inf, 2000, 2.0]))
            A_opt, Ea_opt, n_opt = params_otimizados
            exibir_mensagem(f"Parâmetros calibrados: A={A_opt:.2e}, Ea={Ea_opt:.2f} kJ/mol, n={n_opt:.2f}")
        except Exception as e:
            exibir_erro(f"Falha na calibração do modelo. Tente ajustar os dados de entrada ou os parâmetros iniciais. Erro: {e}")
            return False

        exibir_mensagem("\nEtapa 2: Gerando ensaios sintéticos...")

        df_calibracao_final = df_calibracao.copy()
        df_calibracao_final['Ensaio'] = 0
        lista_de_datasets = [df_calibracao_final]

        fatores_taxa = np.linspace(0.5, 1.5, total_ensaios)[1:] # Gera N-1 fatores

        for i, fator in enumerate(fatores_taxa):
            nova_taxa_C_s = taxa_aquecimento_K_s * fator
            exibir_mensagem(f"  Simulando Ensaio {i+2} com taxa de {nova_taxa_C_s*60:.2f} °C/min...")

            t_sintetico = t_real.copy()
            T_sintetico_C = T_real_C[0] + nova_taxa_C_s * t_sintetico

            rho_sintetico_pct = modelo_gomez_hotza((t_sintetico, T_sintetico_C), A_opt, Ea_opt, n_opt)

            df_sintetico = pd.DataFrame({time_col: t_sintetico, temp_col: T_sintetico_C, dens_col: rho_sintetico_pct, 'Ensaio': i + 1})
            lista_de_datasets.append(df_sintetico)

        exibir_mensagem("\nEtapa 3: Consolidando dataset híbrido.")
        self.df_final_para_analise = pd.concat(lista_de_datasets, ignore_index=True)
        return True

    def get_final_dataframe(self):
        return self.df_final_para_analise

In [ ]:
# modulo_roteadormetodo.py
# Exemplo de Módulo final do seu notebook

import ipywidgets as widgets
from IPython.display import display, clear_output

# Texto de apresentação - somente será exibido agora
intro_text = """
<div style="font-family: Arial, sans-serif; line-height: 1.5;">
    <h2>Final Steps - Ogum Sintering</h2>
    <p>
        Obrigado por utilizar o Ogum Sintering até aqui.
        Agora você pode escolher entre os métodos finais
        para realizar a análise de dados. O Cálculo de LogTheta,
        de acordo com os trabalhos de Su e Johnson para Curvas mestras
        ou a Confecção de Curvas de Arhhenius:
    </p>
</div>
"""

# RadioButtons para escolha do método
method_radio = widgets.RadioButtons(
    options=["LogTheta ( Curvas Mestras )", "Curvas de Arrhenius "],
    value="LogTheta ( Curvas Mestras )",
    description="Method:",
    style={'description_width': 'initial'}
)


# Botão para prosseguir
proceed_button = widgets.Button(description="Realizar Ação", button_style='success')

# Área de saída para logs
out = widgets.Output()

def on_proceed_clicked(b):
    with out:
        clear_output()
        choice = method_radio.value
        if choice == "LogTheta ( Curvas Mestras )":
            print("[INFO] User chose LogTheta method.")
            # Chame aqui o(s) módulo(s) correspondentes ao LogTheta
            # Por exemplo: iniciar_modulo_logtheta()
        else:
            print("[INFO] User chose Arrhenius method.")
            # Chame aqui o(s) módulo(s) correspondentes ao Arrhenius
            # Por exemplo: iniciar_modulo_arrhenius()
    # Opcional: Oculta a interface de escolha após prosseguir
    ui.layout.display = 'none'


proceed_button.on_click(on_proceed_clicked)

# Monta a interface final
ui = widgets.VBox([
    widgets.HTML(value=intro_text),
    method_radio,
    proceed_button,
    out
])

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List

# Importações do Módulo 1
from modulo1_interface import (
    SinteringDataRecord,
    EaSelectionWidget,
    exibir_mensagem,
    exibir_erro,
    gerar_link_download,
    R,
    cumtrapz
)

class ModuloLogTheta:
    """
    Classe refatorada e simplificada para o cálculo de log(theta).
    Usa o widget reutilizável EaSelectionWidget para a definição das energias.
    """
    def __init__(self, dfs_ensaios: List[pd.DataFrame]):
        self.dfs_ensaios = [df for df in dfs_ensaios if df is not None and not df.empty]
        self.n_ensaios = len(self.dfs_ensaios)
        self.sintering_records: List[SinteringDataRecord] = []

        # Widget para seleção de Ea
        self.ea_selector = EaSelectionWidget()
        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        self.btn_calc_logtheta = widgets.Button(description="Calcular LogTheta", button_style='success')
        self.btn_calc_logtheta.on_click(self._on_calc_logtheta)

        self.btn_plot_curves = widgets.Button(description="Visualizar Curvas", button_style='warning')
        self.btn_plot_curves.on_click(self._on_plot_curves)

        self.btn_export_logtheta = widgets.Button(description="Exportar Resultados", button_style='info')
        self.btn_export_logtheta.on_click(self._on_export_logtheta)

        self.ui = widgets.VBox([
            widgets.HTML("<h3>Módulo 4: Cálculo de LogTheta</h3>"),
            self.ea_selector.ui,
            widgets.HBox([self.btn_calc_logtheta, self.btn_plot_curves, self.btn_export_logtheta]),
            self.out
        ])

    def _on_calc_logtheta(self, b):
        with self.out:
            clear_output()
        self.sintering_records = []

        eas = self.ea_selector.get_ea_list()
        if not eas:
            return

        exibir_mensagem(f"Iniciando cálculo... Processando {len(eas)} energias de ativação.")

        for i, df_ensaio in enumerate(self.dfs_ensaios):
            time_col = next((c for c in df_ensaio.columns if c.startswith('Time_s')), None)
            temp_col = next((c for c in df_ensaio.columns if c.startswith('Temperature_C')), None)
            dens_col = next((c for c in df_ensaio.columns if c.startswith('DensidadePct')), None)

            if not (time_col and temp_col and dens_col):
                exibir_erro(f"Ensaio {i+1}: Colunas essenciais não encontradas. Pulando.")
                continue

            if df_ensaio[time_col].size < 2:
                exibir_erro(f"Ensaio {i+1}: Dados insuficientes para integração. Pulando.")
                continue

            T_k = df_ensaio[temp_col].values + 273.15

            for Ea_kj in eas:
                try:
                    Ea_j = Ea_kj * 1000.0
                    theta_inst = (1.0 / T_k) * np.exp(-Ea_j / (R * T_k))
                    integrated = cumtrapz(theta_inst, df_ensaio[time_col].values, initial=0)

                    with np.errstate(divide='ignore', invalid='ignore'):
                        log_integrated = np.log10(integrated)
                    log_integrated[~np.isfinite(log_integrated)] = np.nan

                    # Monta DataFrame incluindo tempo original
                    df_record = pd.DataFrame({
                        'logtheta': log_integrated,
                        'valor': df_ensaio[dens_col].values,
                        'tempo_s': df_ensaio[time_col].values
                    })

                    record = SinteringDataRecord(
                        ensaio_id=i,
                        Ea=Ea_kj,
                        tipo_dado_y='densidade_original',
                        df=df_record
                    )
                    self.sintering_records.append(record)

                except Exception as e:
                    exibir_erro(f"Ensaio {i+1}: Falha ao calcular para Ea={Ea_kj:.2f}. Erro: {e}")

        exibir_mensagem(f"Cálculo concluído. {len(self.sintering_records)} registros gerados.")

    def _on_plot_curves(self, b):
        with self.out:
            clear_output()
        if not self.sintering_records:
            exibir_erro("Nenhum dado calculado para plotar. Execute o cálculo primeiro.")
            return

        from collections import defaultdict
        records_por_ensaio = defaultdict(list)
        for rec in self.sintering_records:
            records_por_ensaio[rec.ensaio_id].append(rec)

        for ensaio_id, records in records_por_ensaio.items():
            fig, ax = plt.subplots(figsize=(8, 6))
            ax.set_title(f"Ensaio {ensaio_id+1} – Densidade vs. LogTheta")
            ax.set_xlabel("log(theta)")
            ax.set_ylabel("Densidade (%)")

            for rec in records:
                df_plot = rec.df.dropna(subset=['logtheta', 'valor'])
                if len(df_plot) < 2:
                    continue
                label = f"Ea={rec.Ea:.2f} kJ/mol"
                ax.plot(df_plot['logtheta'], df_plot['valor'], 'o-', ms=3, label=label)

            ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            ax.grid(True)
            plt.tight_layout()
            plt.show()

        exibir_mensagem("Visualização concluída.")

    def _on_export_logtheta(self, b):
        with self.out:
            clear_output()
        if not self.sintering_records:
            exibir_erro("Nenhum dado disponível para exportação. Execute o cálculo primeiro.")
            return

        all_dfs = []
        for rec in self.sintering_records:
            nome = f"E{rec.ensaio_id+1}_Ea{rec.Ea:.2f}"
            df_sub = rec.df[['logtheta', 'valor']].copy()
            df_sub.columns = [f"logtheta_{nome}", f"densidade_{nome}"]
            all_dfs.append(df_sub)

        df_export = pd.concat(all_dfs, axis=1)
        link = gerar_link_download(df_export, nome_arquivo="Resultados_LogTheta_Largo.xlsx")
        display(link)
        exibir_mensagem("Exportação concluída.")

    def get_sintering_records(self) -> List[SinteringDataRecord]:
        return self.sintering_records

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
from scipy.interpolate import interp1d
from collections import defaultdict
from typing import List

from modulo1_interface import SinteringDataRecord, exibir_mensagem, exibir_erro

class Modulo5_1Alinhamento:
    """
    Módulo refatorado para alinhar curvas de sinterização, preservando a informação de tempo.
    """
    def __init__(self, sintering_records: List[SinteringDataRecord]):
        self.input_records = [rec for rec in sintering_records if rec.tipo_dado_y == 'densidade_original']
        self.all_input_records = sintering_records
        self.aligned_records: List[SinteringDataRecord] = []
        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        if not self.input_records:
            self.main_ui = widgets.HTML("<b>Nenhum dado de 'densidade_original' disponível para alinhamento.</b>")
            return

        self.radio_axis = widgets.RadioButtons(
            options=[("Eixo comum de log(θ)", "logtheta"), ("Eixo comum de Densidade", "dens")],
            value="dens"
        )
        self.int_n_points = widgets.IntText(value=200, description="Nº de Pontos:")
        self.btn_apply = widgets.Button(description="Aplicar Alinhamento", button_style='success', icon='cogs')
        self.btn_apply.on_click(self._on_apply_alignment)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.1 – Alinhamento e Interpolação</h3>"),
            widgets.HTML("<p>Selecione o eixo para criar uma grade comum. As curvas serão reamostradas nesta grade.</p>"),
            widgets.HTML("<b>Eixo de Alinhamento:</b>"),
            self.radio_axis,
            self.int_n_points,
            self.btn_apply,
            self.out
        ])

    def display(self):
        display(self.main_ui)

    def _on_apply_alignment(self, b):
        with self.out:
            clear_output()
            exibir_mensagem("Iniciando processo de alinhamento...")

        self.aligned_records = []
        mode = self.radio_axis.value
        n_points = self.int_n_points.value
        if n_points < 2:
            exibir_erro("Número de pontos deve ser no mínimo 2.")
            return

        for rec in self.input_records:
            df = rec.df.dropna().copy()
            if len(df) < 2 or 'tempo_s' not in df.columns:
                exibir_erro(f"Ensaio {rec.ensaio_id+1}, Ea={rec.Ea}: Dados insuficientes ou sem coluna 'tempo_s'. Pulando.")
                continue

            try:
                if mode == "logtheta":
                    x_axis, y_axis, t_axis = 'logtheta', 'valor', 'tempo_s'
                    x_label, y_label, t_label = 'logtheta_alinhado', 'densidade_alinhada', 'tempo_s_alinhado'
                else:
                    x_axis, y_axis, t_axis = 'valor', 'logtheta', 'tempo_s'
                    x_label, y_label, t_label = 'densidade_alinhada', 'logtheta_alinhado', 'tempo_s_alinhado'

                df.sort_values(x_axis, inplace=True)
                df.drop_duplicates(subset=[x_axis], inplace=True)
                if len(df) < 2:
                    continue

                x_min, x_max = df[x_axis].min(), df[x_axis].max()
                if np.isclose(x_min, x_max):
                    continue

                new_x_grid = np.linspace(x_min, x_max, n_points)
                interp_y = interp1d(df[x_axis], df[y_axis], kind='linear', bounds_error=False, fill_value=np.nan)
                # Cria interpolador para o tempo
                interp_t = interp1d(df[x_axis], df[t_axis], kind='linear', bounds_error=False, fill_value=np.nan)

                df_aligned = pd.DataFrame({
                    x_label: new_x_grid,
                    y_label: interp_y(new_x_grid),
                    t_label: interp_t(new_x_grid)
                }).dropna()

                if df_aligned.empty:
                    continue

                aligned_rec = SinteringDataRecord(
                    ensaio_id=rec.ensaio_id,
                    Ea=rec.Ea,
                    tipo_dado_y='dados_alinhados',
                    df=df_aligned,
                    metadata={'alignment_axis': mode}
                )
                self.aligned_records.append(aligned_rec)

            except Exception as e:
                exibir_erro(f"Falha ao alinhar Ensaio {rec.ensaio_id+1} para Ea={rec.Ea}: {e}")

        exibir_mensagem(f"Alinhamento concluído. {len(self.aligned_records)} registros de 'dados_alinhados' gerados.")

    def get_aligned_records(self) -> List[SinteringDataRecord]:
        return self.all_input_records + self.aligned_records


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List

from modulo1_interface import (
    SinteringDataRecord,
    exibir_mensagem,
    exibir_erro,
    gerar_link_download,
    DataHistory
)

class Modulo5_2BlaineParameters:
    """
    Submódulo 5.2 – Cálculo de Parâmetros de Densificação (Psi e Phi),
    preservando a informação de tempo alinhado.
    """
    def __init__(self, sintering_records: List[SinteringDataRecord]):
        # Todos os records de entrada
        self.input_records = sintering_records
        # Apenas aqueles que vieram do alinhamento
        self.records_to_process = [
            rec for rec in sintering_records if rec.tipo_dado_y == 'dados_alinhados'
        ]
        self.processed_records: List[SinteringDataRecord] = []
        self.history = DataHistory()
        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        if not self.records_to_process:
            self.main_ui = widgets.HTML(
                "<b>Nenhum dado do tipo 'dados_alinhados' encontrado para processar.</b>"
            )
            return

        self.float_rho0 = widgets.FloatText(
            value=55.0,
            description="ρ0 (Dens. Inicial %):",
            tooltip="Densidade inicial da peça verde em porcentagem (ex: 55.0)."
        )
        self.btn_calc = widgets.Button(
            description="Calcular Parâmetros (Psi, Phi)",
            button_style='success',
            icon='calculator'
        )
        self.btn_plot = widgets.Button(
            description="Visualizar Curvas (Psi, Phi)",
            button_style='warning',
            icon='line-chart'
        )

        self.btn_calc.on_click(self._on_calc_blaine)
        self.btn_plot.on_click(self._on_plot)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.2 – Parâmetros de Densificação (Blaine)</h3>"),
            widgets.HTML(
                "<p>Defina a densidade inicial (ρ0) e clique em 'Calcular' para gerar os parâmetros ψ e φ.</p>"
            ),
            self.float_rho0,
            widgets.HBox([self.btn_calc, self.btn_plot]),
            self.out
        ])

    def display(self):
        display(self.main_ui)

    def _on_calc_blaine(self, b):
        with self.out:
            clear_output()
            exibir_mensagem("Iniciando cálculo dos parâmetros de densificação...")

        rho0 = self.float_rho0.value
        if not (0 < rho0 < 100):
            exibir_erro("O valor de ρ0 (Densidade Inicial) deve estar entre 0 e 100.")
            return

        self.processed_records = []
        for rec in self.records_to_process:
            try:
                df_input = rec.df
                required = ['densidade_alinhada', 'logtheta_alinhado', 'tempo_s_alinhado']
                if not all(col in df_input.columns for col in required):
                    exibir_erro(
                        f"Ensaio {rec.ensaio_id+1}, Ea={rec.Ea}: Faltam colunas alinhadas. Pulando."
                    )
                    continue

                dens = df_input['densidade_alinhada'].values
                logtheta = df_input['logtheta_alinhado'].values
                tempo = df_input['tempo_s_alinhado'].values

                # ----- Cálculo de Psi -----
                psi_vals = (dens - rho0) / (100.0 - rho0)
                df_psi = pd.DataFrame({
                    'logtheta': logtheta,
                    'valor': psi_vals,
                    'tempo_s_alinhado': tempo
                })
                self.processed_records.append(
                    SinteringDataRecord(
                        ensaio_id=rec.ensaio_id,
                        Ea=rec.Ea,
                        tipo_dado_y='psi',
                        df=df_psi,
                        metadata=rec.metadata.copy()
                    )
                )
                self.history.push(df_psi, module_name=f"Module 5.2: Psi Ensaio {rec.ensaio_id+1} Ea{rec.Ea}")

                # ----- Cálculo de Phi -----
                phi_vals = (dens - rho0) / (100.0 - dens + 1e-9)
                df_phi = pd.DataFrame({
                    'logtheta': logtheta,
                    'valor': phi_vals,
                    'tempo_s_alinhado': tempo
                })
                self.processed_records.append(
                    SinteringDataRecord(
                        ensaio_id=rec.ensaio_id,
                        Ea=rec.Ea,
                        tipo_dado_y='phi',
                        df=df_phi,
                        metadata=rec.metadata.copy()
                    )
                )
                self.history.push(df_phi, module_name=f"Module 5.2: Phi Ensaio {rec.ensaio_id+1} Ea{rec.Ea}")

            except Exception as e:
                exibir_erro(f"Erro ao calcular Blaine para Ea={rec.Ea}: {e}")

        exibir_mensagem(
            f"Cálculo concluído. {len(self.processed_records)} registros ('psi' e 'phi') gerados."
        )

    def _on_plot(self, b):
        with self.out:
            clear_output()
            if not self.processed_records:
                exibir_erro("Nenhum parâmetro calculado para plotar.")
                return

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=False)
        fig.suptitle("Parâmetros de Densificação vs. log(θ)")

        ax1.set_title("Parâmetro ψ (Psi)")
        ax1.set_xlabel("log(theta)")
        ax1.set_ylabel("ψ = (ρ-ρ₀)/(100-ρ₀)")

        ax2.set_title("Parâmetro φ (Phi)")
        ax2.set_xlabel("log(theta)")
        ax2.set_ylabel("φ = (ρ-ρ₀)/(100-ρ)")

        for rec in self.processed_records:
            dfp = rec.df.dropna()
            if dfp.empty:
                continue
            label = f"E{rec.ensaio_id+1} Ea={rec.Ea:.1f}"
            if rec.tipo_dado_y == 'psi':
                ax1.plot(dfp['logtheta'], dfp['valor'], 'o-', ms=3, label=label)
            elif rec.tipo_dado_y == 'phi':
                ax2.plot(dfp['logtheta'], dfp['valor'], 'o-', ms=3, label=label)

        ax1.grid(True)
        ax1.legend()
        ax2.grid(True)
        ax2.legend()
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

    def get_blaine_records(self) -> List[SinteringDataRecord]:
        """
        Retorna todos os records originais mais os processados (psi e phi).
        """
        return self.input_records + self.processed_records


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd
from typing import List, Dict

# Importações do core/modulo1
from core import (
    SinteringDataRecord, exibir_mensagem, exibir_erro,
    boltzmann_sigmoid, generalized_logistic_stable
)
from scipy.optimize import curve_fit

class Modulo5_3Sigmoides:
    """
    Submódulo 5.3 – Ajuste Sigmoidal.
    Apresenta uma UI para o usuário selecionar quais dados processados (ex: psi, phi)
    devem ser usados para o ajuste com os modelos sigmodais.
    """
    def __init__(self, sintering_records: List[SinteringDataRecord]):
        self.input_records = sintering_records
        self.processed_records: List[SinteringDataRecord] = []
        self._build_ui()

    def _build_ui(self):
        # Mapeia nomes internos para nomes amigáveis na UI
        self.DATA_OPTIONS_MAP = {
            "Dados Alinhados": "dados_alinhados",
            "Parâmetro Psi (ψ)": "psi",
            "Parâmetro Phi (φ)": "phi",
        }

        # Filtra opções para mostrar apenas os tipos de dados que realmente existem
        available_types = set(rec.tipo_dado_y for rec in self.input_records)
        available_options = [ui_text for ui_text, internal in self.DATA_OPTIONS_MAP.items() if internal in available_types]

        if not available_options:
            self.main_ui = widgets.HTML("<b>Nenhum dado compatível (alinhado, psi, phi) encontrado para ajuste.</b>")
            return

        self.select_data = widgets.SelectMultiple(
            options=available_options,
            value=[available_options[0]] if available_options else [], # Seleciona o primeiro por padrão
            description="Ajustar curvas de:",
            rows=len(available_options),
            style={'description_width': 'initial'},
            disabled=False
        )

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Módulo 5.3 – Seleção para Ajuste Sigmoidal</h3>"),
            widgets.HTML("<p>Selecione um ou mais conjuntos de dados para realizar o ajuste com as funções de Boltzmann (4 parâmetros) e Logística Generalizada (5 parâmetros).</p>"),
            self.select_data
        ])

    def display(self):
        display(self.main_ui)

    def run_all_fits(self, data_types_to_fit: List[str], p0_overrides: Dict = None):
        """Executa ambos os ajustes para os tipos de dados informados."""
        exibir_mensagem(f"Iniciando ajuste duplo para: {data_types_to_fit}")
        self.processed_records = []

        records_to_process = [rec for rec in self.input_records if rec.tipo_dado_y in data_types_to_fit]
        if not records_to_process:
            exibir_erro("Nenhum record correspondente à seleção foi encontrado para ajuste."); return

        for rec in records_to_process:
            # ... (o resto da lógica de `run_all_fits` que você já tinha)
            # A lógica interna de ajuste já estava boa, o problema era a falta da UI para a seleção.
            df_clean = rec.df.dropna()
            if len(df_clean) < 5: continue

            x_col = next((c for c in df_clean.columns if 'logtheta' in c), None)
            y_col = next((c for c in df_clean.columns if c != x_col and 'tempo' not in c), None)
            if not x_col or not y_col: continue

            x_data, y_data = df_clean[x_col].values, df_clean[y_col].values

            # Ajuste Boltzmann
            try:
                p0b_manual = p0_overrides.get((rec.ensaio_id, rec.Ea, rec.tipo_dado_y), {}).get('boltzmann') if p0_overrides else None
                p0b = p0b_manual or [y_data.max(), y_data.min(), np.median(x_data), (x_data.max()-x_data.min())/4]
                popt, pcov = curve_fit(boltzmann_sigmoid, x_data, y_data, p0=p0b, maxfev=10000)
                std_err = np.sqrt(np.diag(pcov))
                df_fit = pd.DataFrame({'logtheta_fit': x_data, 'valor_ajustado': boltzmann_sigmoid(x_data, *popt)})
                self.processed_records.append(SinteringDataRecord(
                    rec.ensaio_id, rec.Ea, f"fit_boltzmann_{rec.tipo_dado_y}", df_fit,
                    {'model_name': 'boltzmann', 'popt': popt.tolist(), 'std_errors': std_err.tolist(), 'param_names': ['A1','A2','x0','dx'], 'source_type': rec.tipo_dado_y}
                ))
            except Exception as e:
                exibir_erro(f"Boltzmann falhou para {rec.tipo_dado_y} (Ensaio {rec.ensaio_id+1}, Ea={rec.Ea}): {e}")

            # Ajuste Logístico
            try:
                p0l_manual = p0_overrides.get((rec.ensaio_id, rec.Ea, rec.tipo_dado_y), {}).get('logistic') if p0_overrides else None
                p0l = p0l_manual or [y_data.max(), y_data.min(), np.median(x_data), (x_data.max()-x_data.min())/4, 1.0]
                popt, pcov = curve_fit(generalized_logistic_stable, x_data, y_data, p0=p0l, maxfev=10000)
                std_err = np.sqrt(np.diag(pcov))
                df_fit = pd.DataFrame({'logtheta_fit': x_data, 'valor_ajustado': generalized_logistic_stable(x_data, *popt)})
                self.processed_records.append(SinteringDataRecord(
                    rec.ensaio_id, rec.Ea, f"fit_logistic_{rec.tipo_dado_y}", df_fit,
                    {'model_name': 'logistic', 'popt': popt.tolist(), 'std_errors': std_err.tolist(), 'param_names': ['A1','A2','x0','b','c'], 'source_type': rec.tipo_dado_y}
                ))
            except Exception as e:
                exibir_erro(f"Logístico falhou para {rec.tipo_dado_y} (Ensaio {rec.ensaio_id+1}, Ea={rec.Ea}): {e}")

    def get_all_records_after_fit(self) -> List[SinteringDataRecord]:
        return self.input_records + self.processed_records

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import numpy as np
import pandas as pd
from typing import List, Dict, Any
from collections import defaultdict

# Importando do seu arquivo core
from core import SinteringDataRecord, exibir_mensagem, exibir_erro

class Modulo5_3_1_Revisao:
    """
    Módulo 5.3.1 – Revisão e Refinamento dos Parâmetros de Ajuste.

    Fornece uma interface para:
    1. Visualizar os parâmetros médios obtidos nos ajustes.
    2. Comparar o ponto de inflexão do ajuste (x0) com o experimental.
    3. Opcionalmente, entrar em modo de edição para refinar os parâmetros iniciais (p0) e re-ajustar.
    4. Confirmar os parâmetros e prosseguir.
    """
    def __init__(self, all_records: List[SinteringDataRecord], controller=None):
        self.all_records = all_records
        self.controller = controller

        # Separa os records para fácil acesso
        self.fit_records = [rec for rec in self.all_records if rec.tipo_dado_y.startswith('fit_')]
        self.source_records = {
            (rec.ensaio_id, rec.Ea, rec.tipo_dado_y): rec
            for rec in self.all_records if not rec.tipo_dado_y.startswith('fit_')
        }

        self.manual_widgets: Dict[tuple, Dict[str, Dict[str, widgets.FloatText]]] = {}
        self.out = widgets.Output()

        self._build_ui()
        self._update_summary_table()  # Popula a tabela na inicialização

    def _build_ui(self):
        self.summary_container = widgets.VBox([])

        self.btn_confirm = widgets.Button(description="Confirmar e Prosseguir", button_style='success', icon='check')
        self.btn_manual_toggle = widgets.Button(description="Editar Parâmetros Iniciais (p0)", button_style='info', icon='edit')

        self.manual_adjust_container = widgets.VBox(layout={'display': 'none'})

        self.btn_confirm.on_click(self._on_confirm_and_proceed)
        self.btn_manual_toggle.on_click(self._on_toggle_manual_adjust)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.3.1 – Revisão e Validação dos Ajustes</h3>"),
            widgets.HTML("<p>A tabela abaixo compara os parâmetros médios do ajuste com o ponto de inflexão experimental (pico da taxa de densificação). Verifique a consistência dos valores antes de prosseguir.</p>"),
            self.summary_container,
            widgets.HBox([self.btn_manual_toggle, self.btn_confirm]),
            self.manual_adjust_container,
            self.out
        ])

    def display(self):
        display(self.main_ui)

    def _calculate_experimental_inflexion(self) -> Dict[tuple, float]:
        """Calcula o log(theta) no ponto de inflexão experimental para cada curva de origem."""
        exp_inflexion_points: Dict[tuple, float] = {}

        for key, rec in self.source_records.items():
            df = rec.df.dropna().copy()
            if 'valor' not in df.columns or 'logtheta' not in df.columns or len(df) < 3:
                continue

            # Garante que os dados estejam ordenados por logtheta para o gradiente
            df = df.sort_values('logtheta').reset_index(drop=True)

            # Calcula a derivada e encontra o índice do pico
            taxa = np.gradient(df['valor'], df['logtheta'])
            if len(taxa) > 0:
                idx_pico = int(np.argmax(taxa))
                exp_inflexion_points[key] = float(df['logtheta'].iloc[idx_pico])

        return exp_inflexion_points

    def _update_summary_table(self):
        """Calcula os parâmetros médios e experimentais e exibe na tabela."""
        with self.out:
            clear_output(wait=True)

        if not self.fit_records:
            self.summary_container.children = [widgets.HTML("<p style='color:red;'>Nenhum resultado de ajuste encontrado.</p>")]
            return

        exp_inflexion_points = self._calculate_experimental_inflexion()

        summary_data = []
        for rec in self.fit_records:
            source_type = rec.metadata.get('source_type')
            if not source_type: continue

            key = (rec.ensaio_id, rec.Ea, source_type)
            inflexion_exp = exp_inflexion_points.get(key, np.nan)

            row = {
                'Modelo': rec.metadata.get('model_name', 'N/A'),
                'Dado Origem': source_type,
                'x0 (Ajuste)': rec.metadata.get('popt', [np.nan]*5)[2], # x0 é sempre o 3º parâmetro
                'x0 (Experimental)': inflexion_exp
            }
            summary_data.append(row)

        df_summary = pd.DataFrame(summary_data)
        df_avg = df_summary.groupby(['Modelo', 'Dado Origem']).mean().reset_index()

        html_table = df_avg.to_html(index=False, float_format='{:.3f}'.format, border=0, classes='table table-striped')
        self.summary_container.children = [widgets.HTML(html_table)]

    def _on_toggle_manual_adjust(self, b):
        is_hidden = self.manual_adjust_container.layout.display == 'none'
        self.manual_adjust_container.layout.display = '' if is_hidden else 'none'
        b.description = "Ocultar Ajuste Manual" if is_hidden else "Editar Parâmetros Iniciais (p0)"
        if is_hidden and not self.manual_adjust_container.children:
            self._build_manual_adjust_ui()

    def _build_manual_adjust_ui(self):
        """CORRIGIDO: Cria a interface para edição manual, agrupando por curva."""
        records_by_curve: Dict[tuple, List] = defaultdict(list)
        for rec in self.fit_records:
            source_type = rec.metadata.get('source_type')
            if source_type:
                key = (rec.ensaio_id, rec.Ea, source_type)
                records_by_curve[key].append(rec)

        accordion_children = []
        self.manual_widgets = {}

        # Itera sobre cada CURVA
        for key in sorted(records_by_curve.keys()):
            self.manual_widgets[key] = {}
            # Este VBox conterá os widgets para AMBOS os modelos (Boltzmann e Logistic)
            curve_panel_vbox = widgets.VBox(layout={'border': '1px solid #ddd', 'padding': '10px', 'margin-bottom': '10px'})

            # Itera sobre os MODELOS ajustados para esta curva
            for fit_rec in sorted(records_by_curve[key], key=lambda r: r.metadata.get('model_name', '')):
                model_name = fit_rec.metadata.get('model_name')
                param_names = fit_rec.metadata.get('param_names', [])
                popt = fit_rec.metadata.get('popt', [])

                if not model_name or not param_names: continue

                self.manual_widgets[key][model_name] = {}
                model_widgets = [widgets.HTML(f"<b>Parâmetros Iniciais (p0) para '{model_name.title()}':</b>")]

                for i, name in enumerate(param_names):
                    widget = widgets.FloatText(value=popt[i], description=f"{name}:")
                    model_widgets.append(widget)
                    self.manual_widgets[key][model_name][name] = widget

                curve_panel_vbox.children += tuple(model_widgets)

            accordion_children.append(curve_panel_vbox)

        accordion = widgets.Accordion(children=accordion_children, selected_index=None)
        for i, key in enumerate(sorted(records_by_curve.keys())):
            ensaio_id, Ea, source_type = key
            accordion.set_title(i, f"Ensaio {ensaio_id+1} | Ea={Ea:.2f} | Dado: {source_type}")

        self.btn_refit = widgets.Button(description="Salvar e Re-ajustar com p0 Manuais", button_style="danger", icon='cogs')
        self.btn_refit.on_click(self._on_refit)

        self.manual_adjust_container.children = [
            widgets.HTML("<hr><h4>Edição Manual de Parâmetros Iniciais (p0)</h4>"),
            accordion, self.btn_refit
        ]

    def _on_refit(self, b):
        with self.out:
            clear_output(wait=True)
            exibir_mensagem("Coletando parâmetros manuais para re-ajuste...")

        new_p0_overrides = {}
        for key, models in self.manual_widgets.items():
            new_p0_overrides[key] = {}
            for model_name, params in models.items():
                # Garante que os nomes dos parâmetros no widget correspondem aos necessários pelo modelo
                new_p0_overrides[key][model_name] = [widget.value for widget in params.values()]

        if self.controller and hasattr(self.controller, 'trigger_refit'):
            self.controller.trigger_refit(new_p0_overrides)
        else:
            with self.out: exibir_erro("Controlador ou método 'trigger_refit' não encontrado.")

    def _on_confirm_and_proceed(self, b):
        with self.out:
            clear_output(wait=True)
            exibir_mensagem("Parâmetros confirmados. Avançando para a análise final...")

        if self.controller and hasattr(self.controller, 'show_module_by_step'):
            self.controller.show_module_by_step('ROTEADOR_5_4')
        else:
             with self.out: exibir_erro("Controlador ou método de navegação não encontrado.")

In [ ]:
# modulo5_4_roteador.py

import ipywidgets as widgets
from IPython.display import display, clear_output
from typing import List

# As importações abaixo são necessárias para o funcionamento.
# Elas devem estar definidas em Módulos anteriores do notebook.
# from core import SinteringDataRecord
# from modulo5_4_1_comparaçao import Modulo5_4_1Comparisons
# from modulo5_4_2_Ref import Modulo5_4_2Ref
# from modulo5_4_3_blaine_linear import Modulo5_4_3BlaineLinear
# from modulo1_interface import exibir_mensagem, exibir_erro

class Modulo5_4_Roteador:
    """
    Módulo 5.4.0 Refatorado – Roteador para os métodos de análise final.

    Apresenta ao usuário as opções de análise final (Comparação, Erro Individual,
    Linearização) e, com base na escolha, instancia e exibe o submódulo correspondente,
    passando a lista completa de SinteringDataRecord.
    """

    def __init__(self, sintering_records: List[SinteringDataRecord], controller=None):
        """
        Construtor ajustado para receber a lista de SinteringDataRecord.

        :param sintering_records: A lista completa de dados processados até esta etapa.
        :param controller: Referência ao MainInteractive para navegação.
        """
        # CORREÇÃO: Garante que o nome da variável está correto.
        self.sintering_records = sintering_records
        self.controller = controller
        self.out = widgets.Output()
        self._build_ui()

# Em Modulo5_4_Roteador

    def _build_ui(self):
        # A definição de 'explanation_text' e 'radio_paths' continua a mesma...
        explanation_text = """...""" # Seu texto de explicação
        self.html_explanation = widgets.HTML(value=explanation_text)

        self.radio_paths = widgets.RadioButtons(
            options=[
                ("1. Comparação de Curvas (MRS/MPCD)", "comparisons"),
                ("2. Erro de Ajuste Individual (MSE)", "individual_error"),
                ("3. Linearização de Blaine", "blaine_linear")
            ],
            value="comparisons",
            # A description longa pode ser removida e colocada em um HTML separado
        )

        # Botões
        self.btn_proceed = widgets.Button(description="Executar Análise", button_style='success')
        self.btn_proceed.on_click(self._on_proceed_clicked)
        self.btn_back = widgets.Button(description="Voltar (p/ Módulo 5.3)", button_style='warning')
        self.btn_back.on_click(self._on_back_clicked)

        # --- SOLUÇÃO PARA O LAYOUT ---
        # Colocamos o RadioButtons dentro de uma VBox para garantir a disposição vertical.
        self.main_ui = widgets.VBox([
            self.html_explanation,
            widgets.HTML("<b>Escolha o método de análise final:</b>"),
            widgets.VBox([self.radio_paths]), # <-- MUDANÇA AQUI
            widgets.HBox([self.btn_back, self.btn_proceed]),
            self.out
        ])

    def display(self):
        """Exibe a interface principal do roteador."""
        clear_output(wait=True)
        display(self.main_ui)

    def _on_proceed_clicked(self, b):
        """
        Callback para o botão 'Executar Análise'.
        Instancia o submódulo escolhido, passando a lista de SinteringDataRecord.
        """
        with self.out:
            clear_output()

        choice = self.radio_paths.value

        # CORREÇÃO: Passa 'self.sintering_records' para os construtores dos submódulos
        if choice == "comparisons":
            mod = Modulo5_4_1Comparisons(self.sintering_records, parent=self)
            mod.display()
        elif choice == "individual_error":
            mod = Modulo5_4_2Ref(self.sintering_records, parent=self)
            mod.display()
        elif choice == "blaine_linear":
            mod = Modulo5_4_3BlaineLinear(self.sintering_records, parent=self)
            mod.display()

    def _on_back_clicked(self, b):
        """
        Callback para o botão 'Voltar'.
        Chama um método no controlador principal para navegar para a tela anterior.
        """
        with self.out:
            clear_output()

        if self.controller and hasattr(self.controller, 'show_module_by_step'):
            exibir_mensagem("Retornando ao Módulo 5.3 (Ajuste Sigmoidal)...")
            # O passo 7 no MainInteractive corresponde ao Módulo 5.3
            self.controller.show_module_by_step(7)
        else:
            exibir_erro("Controlador ou método de retorno não foi encontrado.")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.optimize import minimize_scalar
from typing import List
from collections import defaultdict
from pathlib import Path
from uuid import uuid4
import matplotlib.pyplot as plt


# Supondo que as definições de SinteringDataRecord, exibir_mensagem, etc.
# estejam acessíveis a partir de células anteriores.

class Modulo5_4_1Comparisons:
    """
    Submódulo 5.4.1 Modificado – Comparações (MRS e MPCD) para Múltiplos Modelos.

    Consome uma lista de SinteringDataRecord (incluindo resultados de múltiplos ajustes)
    e usa as métricas MRS ou MPCD para encontrar a Energia de Ativação ótima para cada
    conjunto de curvas, permitindo a comparação direta entre os modelos de ajuste.
    Este módulo agora serve como a etapa de "Busca" para o processo de refinamento.
    """

    def __init__(self, sintering_records: List[SinteringDataRecord], parent=None):
        self.all_records = sintering_records
        self.parent = parent
        self.results_df = None
        self.comparison_details_df = None
        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        # <<< LÓGICA DE GERAÇÃO DE OPÇÕES ATUALIZADA >>>
        # Gera as opções de seleção dinamicamente a partir dos records de ajuste disponíveis
        self.DATA_OPTIONS_MAP = {}
        fit_records = [rec for rec in self.all_records if rec.tipo_dado_y.startswith('fit_')]

        for rec in fit_records:
            internal_type = rec.tipo_dado_y
            model_name = rec.metadata.get('model_name', 'desconhecido').title()
            source_type = rec.metadata.get('source_type', 'desconhecido').replace('_', ' ').title()

            # Cria um nome amigável para a UI
            ui_text = f"Ajuste {model_name} ({source_type})"

            if ui_text not in self.DATA_OPTIONS_MAP:
                self.DATA_OPTIONS_MAP[ui_text] = internal_type

        # As opções disponíveis são as chaves únicas do mapa criado
        self.available_options = sorted(list(self.DATA_OPTIONS_MAP.keys()))

        if not self.available_options:
            self.main_ui = widgets.HTML("<b>Nenhum resultado de ajuste compatível com a comparação foi encontrado.</b>")
            return

        self.select_data = widgets.SelectMultiple(
            options=self.available_options,
            description="Comparar curvas de:",
            rows=min(len(self.available_options), 8),
            style={'description_width': 'initial'}
        )

        self.radio_method = widgets.RadioButtons(
            options=[("Mean Residual Square (MRS)", "mrs"), ("Mean Perpendicular Curve Distance (MPCD)", "mpcd")],
            value="mrs", description="Critério Padrão:",
            style={'description_width': 'initial'}
        )

        self.btn_compare = widgets.Button(description="Executar Comparação", button_style='success', icon='calculator')
        self.btn_compare.on_click(self._on_compare)

        # --- NOVOS BOTÕES ADICIONADOS ---
        self.btn_visualize_error = widgets.Button(
            description="Visualizar Curvas de Erro",
            button_style='info',
            icon='line-chart',
            disabled=True
        )
        self.btn_visualize_error.on_click(self._on_visualize_error_curves)

        self.btn_refine = widgets.Button(
            description="Refinar Melhor Resultado",
            button_style='success',
            icon='cogs',
            disabled=True
        )
        self.btn_refine.on_click(self._on_start_refinement)
        # --- FIM DA ADIÇÃO DE BOTÕES ---

        self.btn_export = widgets.Button(description="Exportar Resultados", button_style='info', disabled=True, icon='download')
        self.btn_export.on_click(self._on_export)

        self.btn_back = widgets.Button(description="Voltar", button_style='warning', icon='arrow-left')
        self.btn_back.on_click(self._on_back_clicked)

        # --- LAYOUT ATUALIZADO PARA INCLUIR OS NOVOS BOTÕES ---
        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.4.1 – Comparação e Busca da Melhor Energia de Ativação</h3>"),
            widgets.HTML("<p>Selecione os conjuntos de curvas, execute a comparação e depois visualize o gráfico de erro ou refine o melhor resultado encontrado.</p>"),
            self.select_data,
            self.radio_method,
            widgets.HBox([self.btn_back, self.btn_compare, self.btn_visualize_error, self.btn_refine, self.btn_export]),
            self.out
        ])

    def display(self):
        display(self.main_ui)

    # <<< LÓGICA DE COMPARAÇÃO E RESULTADOS ATUALIZADA >>>
    def _on_compare(self, b):
        with self.out:
            clear_output()
            exibir_mensagem("Iniciando comparação para ambos os critérios (MRS e MPCD)...")

        # Desabilita os botões enquanto o cálculo é feito
        self.btn_export.disabled = True
        self.btn_visualize_error.disabled = True
        self.btn_refine.disabled = True

        selected_ui_options = self.select_data.value
        if not selected_ui_options:
            exibir_erro("Por favor, selecione pelo menos um conjunto de dados para comparar.")
            return

        selected_internal_types = [self.DATA_OPTIONS_MAP[opt] for opt in selected_ui_options]

        # --- LÓGICA MODIFICADA PARA CALCULAR AMBOS OS MÉTODOS ---
        methods_to_run = ["mrs", "mpcd"]
        all_results_summary = []
        all_errors_details = []

        for data_type in selected_internal_types:
            for method in methods_to_run:
                records_to_process = [rec for rec in self.all_records if rec.tipo_dado_y == data_type]
                if not records_to_process: continue

                sample_rec = records_to_process[0]
                model_name = sample_rec.metadata.get('model_name', 'N/A').title()
                source_type = sample_rec.metadata.get('source_type', 'N/A')

                records_by_ea = defaultdict(list)
                for rec in records_to_process:
                    records_by_ea[rec.Ea].append(rec)

                errors_for_this_type = {}
                for Ea, group in records_by_ea.items():
                    if len(group) < 2: continue

                    x_col = next((col for col in group[0].df.columns if 'logtheta' in col), None)
                    if x_col is None: continue

                    curves_data = [rec.df[[x_col, 'valor_ajustado']].dropna().values for rec in group]

                    error_value = np.nan
                    try:
                        if method == "mrs":
                            error_value = self._calculate_mrs_for_ea(curves_data)
                        elif method == "mpcd":
                            error_value = self._calculate_mpcd_for_ea(curves_data)

                        if pd.notna(error_value) and np.isfinite(error_value):
                            errors_for_this_type[Ea] = error_value
                            all_errors_details.append({
                                "Modelo": model_name,
                                "Dado de Origem": source_type,
                                "Metodo": method.upper(),
                                "Ea_kJ_mol": Ea,
                                "Erro": error_value
                            })
                    except Exception as e:
                        exibir_erro(f"Erro ao calcular {method.upper()} para {data_type} com Ea={Ea}: {e}")

                if errors_for_this_type:
                    best_ea = min(errors_for_this_type, key=errors_for_this_type.get)
                    min_error = errors_for_this_type[best_ea]
                    all_results_summary.append({
                        "Modelo": model_name,
                        "Dado de Origem": source_type,
                        "Metodo": method.upper(),
                        "Melhor_Ea_kJ_mol": best_ea,
                        "Erro_Minimo": min_error
                    })

        if all_results_summary:
            self.results_df = pd.DataFrame(all_results_summary)
            self.comparison_details_df = pd.DataFrame(all_errors_details)

            with self.out:
                display(HTML("<h4>Resumo: Melhores Energias de Ativação Encontradas</h4>"))
                display(self.results_df.style.format({'Melhor_Ea_kJ_mol': '{:.2f}', 'Erro_Minimo': '{:.6g}'}))

                display(HTML("<h4>Detalhamento do Erro vs. Ea</h4>"))
                display(self.comparison_details_df.style.format({'Ea


In [ ]:
# modulo5_4_2_Ref.py

import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
from typing import List
from collections import defaultdict

# Importações necessárias do Módulo 1
from modulo1_interface import (
    SinteringDataRecord,
    exibir_mensagem,
    exibir_erro,
    boltzmann_sigmoid
)

class Modulo5_4_2Ref:
    """
    Submódulo 5.4.2 Refatorado – Método de Erro de Ajuste Individual (MSE).

    Encontra a melhor Energia de Ativação (Ea) para cada curva individualmente,
    identificando qual valor de Ea, dentre os já calculados, resulta em uma curva
    com o menor erro quadrático médio (MSE) ao ser ajustada por uma função sigmoidal.
    """
    def __init__(self, sintering_records: List[SinteringDataRecord], parent=None):
        self.all_records = sintering_records
        self.parent = parent
        self.results_df = None
        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        # Mapeamento da UI para os tipos de dados internos
        self.DATA_OPTIONS_MAP = {
            "Densidade Alinhada": "dados_alinhados",
            "Parâmetro Psi (ψ)": "psi",
            "Parâmetro Phi (φ)": "phi",
            "Densidade Original": "densidade_original"
        }

        # Filtra as opções disponíveis conforme registros
        available_types = set(rec.tipo_dado_y for rec in self.all_records)
        options = [ui for ui, itype in self.DATA_OPTIONS_MAP.items() if itype in available_types]

        if not options:
            self.main_ui = widgets.HTML("<b>Nenhum tipo de dado compatível com este método foi encontrado.</b>")
            return

        self.select_data = widgets.Dropdown(
            options=options,
            description="Analisar dados de:",
            style={'description_width': 'initial'}
        )

        self.btn_calc = widgets.Button(
            description="Calcular Melhor Ea (por Erro Individual)",
            button_style='success'
        )
        self.btn_calc.on_click(self._on_calc)

        self.btn_back = widgets.Button(
            description="Voltar",
            button_style='warning'
        )
        self.btn_back.on_click(self._on_back_clicked)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.4.2 – Método de Erro de Ajuste Individual</h3>"),
            widgets.HTML("<p>Este método avalia qual das Energias de Ativação já calculadas produz a curva mais 'perfeita' (com menor erro de auto-ajuste) para cada ensaio.</p>"),
            self.select_data,
            widgets.HBox([self.btn_back, self.btn_calc]),
            self.out
        ])

    def display(self):
        """Exibe a interface principal do módulo."""
        display(self.main_ui)

    def _calc_mse_error(self, x_vals, y_vals):
        """Calcula o erro quadrático médio (MSE) de um ajuste Boltzmann."""
        if len(x_vals) < 4:
            return np.inf
        try:
            p0 = [
                np.max(y_vals),
                np.min(y_vals),
                np.median(x_vals),
                (np.max(x_vals) - np.min(x_vals)) / 4
            ]
            popt, _ = curve_fit(
                boltzmann_sigmoid,
                x_vals,
                y_vals,
                p0=p0,
                maxfev=5000
            )
            y_pred = boltzmann_sigmoid(x_vals, *popt)
            return np.mean((y_vals - y_pred) ** 2)
        except Exception:
            return np.inf

    def _on_calc(self, b):
        with self.out:
            clear_output()
            selected_ui = self.select_data.value
            selected_type = self.DATA_OPTIONS_MAP[selected_ui]

            # Filtra registros conforme tipo selecionado
            records_to_process = [
                rec for rec in self.all_records
                if rec.tipo_dado_y == selected_type
            ]
            if not records_to_process:
                exibir_erro(f"Nenhum dado do tipo '{selected_type}' foi encontrado para análise.")
                return

            results_by_ensaio = defaultdict(dict)
            for rec in records_to_process:
                df_clean = rec.df.dropna()
                x_col = next((c for c in df_clean.columns if 'logtheta' in c), None)
                y_col = next((c for c in df_clean.columns if 'valor' in c), None)
                if x_col is None or y_col is None:
                    continue

                error = self._calc_mse_error(
                    df_clean[x_col].values,
                    df_clean[y_col].values
                )
                if np.isfinite(error):
                    results_by_ensaio[rec.ensaio_id][rec.Ea] = error

            summary_list = []
            for ens_id, ea_errors in results_by_ensaio.items():
                if not ea_errors:
                    continue
                best_ea = min(ea_errors, key=ea_errors.get)
                summary_list.append({
                    "Ensaio_ID": ens_id + 1,
                    "Tipo de Dado": selected_type,
                    "Melhor_Ea (kJ/mol)": best_ea,
                    "Min_MSE": ea_errors[best_ea]
                })

            if summary_list:
                self.results_df = pd.DataFrame(summary_list)
                exibir_mensagem("--- MELHOR Ea POR ENSAIO (MÉTODO DE ERRO INDIVIDUAL) ---")
                display(self.results_df.style.format({
                    'Melhor_Ea (kJ/mol)': '{:.2f}',
                    'Min_MSE': '{:.6g}'
                }))
            else:
                exibir_erro("Não foi possível calcular nenhum resultado válido.")

    def _on_back_clicked(self, b):
        with self.out:
            clear_output()
        if self.parent and hasattr(self.parent, 'display'):
            self.parent.display()
        else:
            exibir_erro("Controlador ou método de retorno não encontrado.")


In [ ]:
# modulo5_4_3_blaine_linear.py

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
from scipy.stats import linregress
import matplotlib.pyplot as plt
from typing import List

from modulo1_interface import SinteringDataRecord, exibir_mensagem, exibir_erro

class Modulo5_4_3BlaineLinear:
    """
    Submódulo 5.4.3 – Método de Linearização de Blaine.
    Aplica uma transformação linearizante a dados de 'psi' ou 'phi' e
    realiza regressão linear para avaliar a qualidade do ajuste (R²).
    """

    def __init__(self, sintering_records: List[SinteringDataRecord], parent=None):
        self.all_records = sintering_records
        self.parent = parent
        self.results_df = None
        self.out = widgets.Output()
        self._build_ui()

    def _build_ui(self):
        # Mapeamento fixo, mas só mostra o que existe nos records
        self.DATA_OPTIONS_MAP = {
            "Parâmetro Psi (ψ)": "psi",
            "Parâmetro Phi (φ)": "phi"
        }
        available = [ui for ui, it in self.DATA_OPTIONS_MAP.items()
                     if any(rec.tipo_dado_y == it for rec in self.all_records)]
        if not available:
            self.main_ui = widgets.HTML("<b>Nenhum dado 'psi' ou 'phi' disponível para linearização.</b>")
            return

        self.select_param = widgets.Dropdown(
            options=available,
            description="Linearizar:",
            style={'description_width': 'initial'}
        )
        self.btn_linearize = widgets.Button(description="Executar Linearização", button_style='success')
        self.btn_linearize.on_click(self._on_linearize)

        self.btn_plot = widgets.Button(description="Visualizar Linearização", button_style='warning')
        self.btn_plot.on_click(self._on_plot)

        self.btn_back = widgets.Button(description="Voltar", button_style='warning')
        self.btn_back.on_click(self._on_back_clicked)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.4.3 – Método de Linearização de Blaine</h3>"),
            widgets.HTML("<p>Transformação e regressão linear em 'psi' ou 'phi', exibindo R².</p>"),
            self.select_param,
            widgets.HBox([self.btn_linearize, self.btn_plot, self.btn_back]),
            self.out
        ])

    def display(self):
        display(self.main_ui)

    def _on_linearize(self, b):
        with self.out:
            clear_output()
        param_choice = self.DATA_OPTIONS_MAP[self.select_param.value]
        records = [rec for rec in self.all_records if rec.tipo_dado_y == param_choice]
        if not records:
            exibir_erro(f"Nenhum dado do tipo '{param_choice}' encontrado para linearizar.")
            return

        results = []
        for rec in records:
            df = rec.df.dropna()
            x_col = next((c for c in df.columns if c.startswith('logtheta')), None)
            y_col = next((c for c in df.columns if c.startswith('valor')), None)
            if x_col is None or y_col is None:
                exibir_erro(f"Ensaio {rec.ensaio_id+1}, Ea={rec.Ea}: colunas não encontradas.")
                continue

            x_vals = df[x_col].values
            y_vals = df[y_col].values

            try:
                if param_choice == 'psi':
                    mask = (y_vals > 1e-9) & (y_vals < 1 - 1e-9)
                    x_t = x_vals[mask]
                    y_t = np.log(y_vals[mask] / (1 - y_vals[mask]))
                else:  # 'phi'
                    mask = y_vals > 1e-9
                    x_t = x_vals[mask]
                    y_t = np.log(y_vals[mask])

                if len(x_t) < 2:
                    exibir_erro(f"Ensaio {rec.ensaio_id+1}, Ea={rec.Ea}: dados insuficientes após máscara.")
                    continue

                slope, intercept, r_value, _, _ = linregress(x_t, y_t)
                results.append({
                    "Ensaio_ID": rec.ensaio_id + 1,
                    "Ea_kJ_mol": rec.Ea,
                    "DataType": param_choice,
                    "R_squared": r_value**2,
                    "Slope": slope,
                    "Intercept": intercept
                })
            except Exception as e:
                exibir_erro(f"Erro na regressão Ensaio {rec.ensaio_id+1}, Ea={rec.Ea}: {e}")

        if results:
            self.results_df = pd.DataFrame(results).sort_values(by="R_squared", ascending=False)
            with self.out:
                exibir_mensagem("Linearização concluída. Resultados ordenados por R²:")
                display(self.results_df.style.format({'R_squared': '{:.4f}', 'Slope': '{:.4f}', 'Intercept': '{:.4f}'}))
        else:
            exibir_erro("Nenhum resultado de linearização pôde ser calculado.")

    def _on_plot(self, b):
        with self.out:
            clear_output()
        if self.results_df is None or self.results_df.empty:
            exibir_erro("Nenhum resultado disponível. Execute 'Executar Linearização' primeiro.")
            return

        param_choice = self.DATA_OPTIONS_MAP[self.select_param.value]
        records = [rec for rec in self.all_records if rec.tipo_dado_y == param_choice]
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.set_title(f"Linearização de {param_choice.upper()} – Dados vs. Ajuste")
        ax.set_xlabel("log(theta)")
        ylabel = "ln(ψ/(1-ψ))" if param_choice == 'psi' else "ln(φ)"
        ax.set_ylabel(ylabel)

        for rec in records:
            df = rec.df.dropna()
            x_col = next((c for c in df.columns if c.startswith('logtheta')), None)
            y_col = next((c for c in df.columns if c.startswith('valor')), None)
            x_vals = df[x_col].values
            y_vals = df[y_col].values

            if param_choice == 'psi':
                mask = (y_vals > 1e-9) & (y_vals < 1 - 1e-9)
                x_t = x_vals[mask]
                y_t = np.log(y_vals[mask] / (1 - y_vals[mask]))
            else:
                mask = y_vals > 1e-9
                x_t = x_vals[mask]
                y_t = np.log(y_vals[mask])

            if len(x_t) < 2:
                continue

            slope = self.results_df.query(
                "Ensaio_ID==@ (rec.ensaio_id+1) & Ea_kJ_mol==@rec.Ea"
            )['Slope'].iloc[0]
            intercept = self.results_df.query(
                "Ensaio_ID==@ (rec.ensaio_id+1) & Ea_kJ_mol==@rec.Ea"
            )['Intercept'].iloc[0]

            y_fit = slope * x_t + intercept
            label = f"E{rec.ensaio_id+1} Ea={rec.Ea:.2f}"
            ax.plot(x_t, y_t, 'o', ms=4, label=f"{label} (dados)")
            ax.plot(x_t, y_fit, '-', label=f"{label} (fit R²={self.results_df.loc[(self.results_df['Ensaio_ID']==rec.ensaio_id+1)&(self.results_df['Ea_kJ_mol']==rec.Ea),'R_squared'].iloc[0]:.3f})")

        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax.grid(True)
        plt.tight_layout()
        plt.show()
        with self.out:
            exibir_mensagem("Visualização de linearização concluída.")

    def _on_back_clicked(self, b):
        with self.out:
            clear_output()
        if self.parent and hasattr(self.parent, 'display'):
            self.parent.display()
        else:
            exibir_erro("Controlador (parent) ou método de retorno não encontrado.")


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import numpy as np
import pandas as pd
from typing import List, Dict, Optional, Callable
from scipy.optimize import least_squares, curve_fit
import matplotlib.pyplot as plt

# Supondo que as definições e funções abaixo foram importadas do seu módulo 'core' ou 'modulo1_interface'
from core import (
    SinteringDataRecord, exibir_mensagem, exibir_erro, R, cumtrapz,
    boltzmann_sigmoid, generalized_logistic_stable
)

class Modulo5_5_Refinamento:
    """
    Módulo 5.5 – Refinamento da Energia de Ativação e Cálculo de Incerteza.

    Este módulo recebe uma estimativa inicial para a Energia de Ativação (Ea)
    e utiliza um método de otimização não linear (mínimos quadrados) para
    encontrar um valor de alta precisão que melhor colapsa as curvas experimentais
    em uma única curva mestra. Ao final, calcula e exibe o intervalo de
    confiança para o valor refinado de Ea.
    """

    def __init__(self,
                 sintering_records: List[SinteringDataRecord],
                 initial_ea: float,
                 target_curve_type: str,
                 parent=None):
        """
        Inicializa o módulo de refinamento.

        Args:
            sintering_records: A lista completa de todos os records.
            initial_ea: A estimativa inicial de Ea (em kJ/mol).
            target_curve_type: O tipo de dado interno da curva a ser refinada (ex: 'fit_boltzmann_psi').
            parent: A referência ao controlador principal (MainInteractive).
        """
        self.all_records = sintering_records
        self.initial_ea = initial_ea
        self.target_curve_type = target_curve_type
        self.parent = parent

        # Atributos para armazenar dados e resultados
        self.final_ea: Optional[float] = None
        self.confidence_interval: Optional[float] = None
        self.objective_function_model: Optional[Callable] = None
        self.source_data_type: Optional[str] = None
        self.source_records: List[SinteringDataRecord] = []
        self.original_exp_records: Dict[int, SinteringDataRecord] = {}

        self.out = widgets.Output()
        self._build_ui()
        self.run_refinement()

    def _prepare_data_and_model(self):
        """
        Prepara os dados de origem, mapeia os dados experimentais originais
        e seleciona o modelo de ajuste com base no tipo de curva alvo.
        """
        # 1. Determina o tipo de dado original (ex: 'psi', 'phi')
        self.source_data_type = self.target_curve_type.replace('fit_boltzmann_', '').replace('fit_logistic_', '')

        # 2. Seleciona a função do modelo sigmoidal a ser usada
        if 'boltzmann' in self.target_curve_type:
            self.objective_function_model = boltzmann_sigmoid
        elif 'logistic' in self.target_curve_type:
            self.objective_function_model = generalized_logistic_stable
        else:
            raise ValueError(f"Tipo de curva alvo desconhecido para refinamento: {self.target_curve_type}")

        # 3. Filtra para obter os records que serão usados como 'y' no ajuste (ex: todos os 'psi')
        self.source_records = [
            rec for rec in self.all_records if rec.tipo_dado_y == self.source_data_type
        ]

        # 4. CRUCIAL: Mapeia os dados experimentais originais (T vs t) por ensaio_id
        # Isso é necessário para recalcular o log(theta) para qualquer valor de Ea.
        self.original_exp_records = {
            rec.ensaio_id: rec for rec in self.all_records
            if rec.tipo_dado_y == 'densidade_original'
        }

        if not self.source_records:
            raise ValueError(f"Nenhum record do tipo '{self.source_data_type}' encontrado para refinamento.")
        if not self.original_exp_records:
            raise ValueError("Nenhum record do tipo 'densidade_original' encontrado. Não é possível recalcular log(theta).")

    def _objective_function(self, ea_array: np.ndarray) -> np.ndarray:
        """
        Função de erro (objetivo) para o otimizador.

        Para uma dada Energia de Ativação (Ea), esta função:
        1. Recalcula o log(theta) para cada curva experimental.
        2. Combina todos os pontos (log(theta), valor) em um único dataset "mestre".
        3. Ajusta uma única curva sigmoidal a este dataset mestre.
        4. Retorna os resíduos (diferenças) entre os pontos e a curva mestra ajustada.

        O otimizador `least_squares` tentará minimizar a soma dos quadrados desses resíduos.
        """
        Ea_kj = ea_array[0]
        Ea_j = Ea_kj * 1000.0  # Converte para J/mol

        all_points_list = []

        # Itera sobre os records de dados de origem (ex: 'psi' ou 'phi')
        for rec in self.source_records:
            ensaio_id = rec.ensaio_id

            # Pega o dado experimental original correspondente para obter T(t)
            original_rec = self.original_exp_records.get(ensaio_id)
            if original_rec is None:
                continue

            # Extrai tempo e temperatura do ensaio original
            time_col = next((c for c in original_rec.df.columns if c.startswith('tempo_s')), None)
            temp_col = next((c for c in original_rec.df.columns if c.startswith('Temperature_C')), None)

            if not time_col or not temp_col:
                continue

            t_vals = original_rec.df[time_col].values
            T_k = original_rec.df[temp_col].values + 273.15

            # Recalcula log(theta) com a nova Ea
            theta_inst = (1.0 / T_k) * np.exp(-Ea_j / (R * T_k))
            integrated = cumtrapz(theta_inst, t_vals, initial=0)
            with np.errstate(divide='ignore', invalid='ignore'):
                log_integrated = np.log10(integrated)

            # Combina o novo log(theta) com o valor correspondente (psi, phi, etc.)
            # e os dados de temperatura para uso posterior.
            df_recalc = pd.DataFrame({
                'logtheta': log_integrated,
                'y_valor': rec.df['valor'].values,
                'Temperature_C': T_k - 273.15  # Armazena T original para referência
            }).dropna()

            all_points_list.append(df_recalc)

        if not all_points_list:
            return np.array([1e6])  # Retorna erro grande se não houver dados

        # Concatena todos os pontos em um único DataFrame mestre
        master_df = pd.concat(all_points_list, ignore_index=True)
        x_data = master_df['logtheta'].values
        y_data = master_df['y_valor'].values

        # Ajusta uma única curva mestra a todos os pontos combinados
        try:
            # Estimativas iniciais robustas para a curva mestra
            p0_A1 = np.max(y_data)
            p0_A2 = np.min(y_data)
            p0_x0 = np.median(x_data)
            p0_dx = (np.max(x_data) - np.min(x_data)) / 4

            p0 = [p0_A1, p0_A2, p0_x0, p0_dx]
            if self.objective_function_model == generalized_logistic_stable:
                p0.append(1.0) # Adiciona p0 para o parâmetro 'c'

            popt, _ = curve_fit(self.objective_function_model, x_data, y_data, p0=p0, maxfev=8000)

            # Retorna os resíduos entre os dados e a curva mestra única
            residuals = y_data - self.objective_function_model(x_data, *popt)
            return residuals
        except RuntimeError:
            # Se o ajuste da curva mestra falhar, penaliza com erro grande
            return np.full_like(y_data, 1e6)

    def run_refinement(self):
        """
        Executa o processo de otimização e cálculo de incerteza.
        """
        with self.out:
            clear_output()
            exibir_mensagem("Executando ajuste de refinamento não linear... Por favor, aguarde.")

        try:
            self._prepare_data_and_model()

            # Executa a otimização de mínimos quadrados
            result = least_squares(
                fun=self._objective_function,
                x0=[self.initial_ea],
                method='trf',  # Trust Region Reflective, bom para problemas com limites
                bounds=([10], [5000])  # Limites razoáveis para Ea em kJ/mol
            )

            self.final_ea = result.x[0]

            # Calcula o intervalo de confiança a partir da matriz Jacobiana
            jac = result.jac
            # Adiciona um pequeno valor à diagonal para estabilidade numérica
            cov_inv = jac.T.dot(jac) + 1e-8 * np.eye(jac.shape[1])
            pcov = np.linalg.inv(cov_inv)
            std_err = np.sqrt(np.diag(pcov))
            # Intervalo de confiança de 95% (z-score ≈ 1.96)
            self.confidence_interval = 1.96 * std_err[0]

        except Exception as e:
            with self.out:
                exibir_erro(f"Ocorreu um erro crítico durante o refinamento: {e}")
            return

        # Atualiza a UI com os resultados
        self._update_results_display()

    def _build_ui(self):
        """Constrói a interface do usuário para este módulo."""
        self.results_container = widgets.VBox([
            widgets.HTML("<h4>Calculando...</h4>")
        ])

        self.btn_back = widgets.Button(description="Voltar para Análise", button_style='warning', icon='arrow-left')
        self.btn_back.on_click(self._on_back_clicked)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Submódulo 5.5 – Refinamento da Energia de Ativação</h3>"),
            self.results_container,
            self.btn_back,
            self.out
        ])

    def _update_results_display(self):
        """Atualiza a interface com os resultados finais do refinamento."""
        if self.final_ea is None or self.confidence_interval is None:
            self.results_container.children = [widgets.HTML("<p style='color:red;'>Falha ao calcular o resultado final.</p>")]
            return

        resultado_html = f"""
        <div style="border: 1px solid #ccc; padding: 15px; border-radius: 5px; background-color: #f9f9f9;">
            <p>A partir de um valor inicial de <b>{self.initial_ea:.2f} kJ/mol</b>, o ajuste fino convergiu para:</p>
            <h1 style="text-align: center; color: #28a745; margin: 10px 0;">
                Ea = {self.final_ea:.2f} &pm; {self.confidence_interval:.2f} kJ/mol
            </h1>
            <p style="text-align: center; font-style: italic;">(Intervalo de Confiança de 95%)</p>
        </div>
        """
        self.results_container.children = [widgets.HTML(resultado_html)]

    def _on_back_clicked(self, b):
        """Navega de volta para o módulo anterior (roteador de análise)."""
        if self.parent and hasattr(self.parent, 'show_module_by_step'):
            # Navega de volta para o passo do roteador de análise final
            self.parent.show_module_by_step('ROTEADOR_5_4')
        else:
            with self.out:
                exibir_erro("Controlador principal ou método de navegação não encontrado.")

    def display(self):
        """Exibe a interface principal do módulo."""
        display(self.main_ui)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d

# Importações de utilitários do Módulo 1
from modulo1_interface import exibir_mensagem, exibir_erro, gerar_link_download


def _calculate_derivatives_robust(df: pd.DataFrame, time_col: str, temp_col: str, dens_col: str):
    """
    Calcula derivadas de T e p em relação ao tempo e à temperatura,
    aplicando filtro Savitzky-Golay para suavização prévia.
    Retorna interpoladores de T(dens), dT/dt(dens) e dp/dT(dens).
    """
    # Ordena e copia
    df_sorted = df.sort_values(time_col).reset_index(drop=True)
    n_pts = len(df_sorted)
    if n_pts < 5:
        raise ValueError("Dados insuficientes (<5 pontos) para cálculo robusto.")

    # Determina janela ímpar para filtro
    max_win = 11
    win = min(max_win, n_pts if n_pts % 2 == 1 else n_pts - 1)
    window_length = win if win >= 5 else (5 if n_pts >= 5 else n_pts)
    if window_length >= 5:
        y_vals = savgol_filter(df_sorted[dens_col].values, window_length, 2)
        T_vals = savgol_filter(df_sorted[temp_col].values + 273.15, window_length, 2)
    else:
        y_vals = df_sorted[dens_col].values
        T_vals = df_sorted[temp_col].values + 273.15

    # Derivadas
    dTdt = np.gradient(T_vals, df_sorted[time_col].values)
    dpdT = np.gradient(y_vals, T_vals)

    # Prepara para interpolação: garante densidade estritamente crescente
    interp_df = pd.DataFrame({
        'dens': y_vals,
        'T': T_vals,
        'dTdt': dTdt,
        'dpdT': dpdT
    }).sort_values('dens').drop_duplicates('dens')

    if len(interp_df) < 2:
        raise ValueError("Dados insuficientes após filtragem para interpolação.")

    f_T = interp1d(interp_df['dens'], interp_df['T'], kind='linear', bounds_error=False, fill_value='extrapolate')
    f_dTdt = interp1d(interp_df['dens'], interp_df['dTdt'], kind='linear', bounds_error=False, fill_value='extrapolate')
    f_dpdT = interp1d(interp_df['dens'], interp_df['dpdT'], kind='linear', bounds_error=False, fill_value='extrapolate')

    return f_T, f_dTdt, f_dpdT


class Modulo6_0_Arrhenius:
    """
    Módulo 6.0 – Cálculo de Curvas de Arrhenius.
    """

    def __init__(self, dfs_ensaios: list):
        self.dfs_ensaios = dfs_ensaios
        self.arrhenius_results = {}

        # Botões
        self.btn_calc = widgets.Button(description="Calcular Arrhenius", button_style='success')
        self.btn_export = widgets.Button(description="Exportar Resultados Arrhenius", button_style='info')
        self.out = widgets.Output()

        self.btn_calc.on_click(self._on_calc)
        self.btn_export.on_click(self._on_export)

        self.main_ui = widgets.VBox([
            widgets.HTML("<h3>Módulo 6.0 – Cálculo de Arrhenius</h3>"),
            widgets.HTML("<p>Calcule ln[(dρ/dt)/T] vs. 1/T para diferentes valores de densidade.</p>"),
            widgets.HBox([self.btn_calc, self.btn_export]),
            self.out
        ])

    def display(self):
        clear_output(wait=True)
        display(self.main_ui)

    def _on_calc(self, _b):
        with self.out:
            clear_output()
            exibir_mensagem("Iniciando cálculo de Arrhenius para todos os ensaios...")

        self.arrhenius_results.clear()

        for idx, df in enumerate(self.dfs_ensaios):
            time_col = next((c for c in df.columns if c.startswith('Time_s')), None)
            temp_col = next((c for c in df.columns if c.startswith('Temperature_C')), None)
            dens_col = next((c for c in df.columns if c.startswith('DensidadePct')), None)

            if not all([time_col, temp_col, dens_col]):
                with self.out:
                    exibir_erro(f"Ensaio {idx+1}: colunas necessárias faltando. Pulando.")
                continue

            df_clean = df.dropna(subset=[time_col, temp_col, dens_col])
            if len(df_clean) < 5:
                with self.out:
                    exibir_erro(f"Ensaio {idx+1}: insuficiente para análise (<5 pontos). Pulando.")
                continue

            dens_min, dens_max = df_clean[dens_col].min(), df_clean[dens_col].max()
            if dens_max <= dens_min:
                with self.out:
                    exibir_erro(f"Ensaio {idx+1}: densidade constante. Pulando.")
                continue

            # Gera alvos de densidade espaçados
            n_points = 10
            dens_targets = np.linspace(
                dens_min + 0.1*(dens_max-dens_min),
                dens_max - 0.1*(dens_max-dens_min),
                n_points
            )

            try:
                f_T, f_dTdt, f_dpdT = _calculate_derivatives_robust(
                    df_clean, time_col, temp_col, dens_col
                )
            except Exception as e:
                with self.out:
                    exibir_erro(f"Ensaio {idx+1}: falha derivadas - {e}")
                continue

            results = []
            for dens in dens_targets:
                T_val = f_T(dens)
                dTdt_val = f_dTdt(dens)
                dpdT_val = f_dpdT(dens)
                if any([not np.isfinite(T_val), not np.isfinite(dTdt_val), not np.isfinite(dpdT_val)]):
                    continue
                if T_val <= 0 or dTdt_val <= 0 or dpdT_val <= 0:
                    continue

                taxa = dpdT_val * dTdt_val
                if taxa <= 0:
                    continue

                ln_term = np.log(taxa / T_val)
                inv_T = 1.0 / T_val
                results.append((dens, inv_T, ln_term))

            self.arrhenius_results[idx] = results

        with self.out:
            exibir_mensagem("Cálculo de Arrhenius concluído para todos os ensaios.")

    def _on_export(self, _b):
        with self.out:
            clear_output()

        if not self.arrhenius_results or not any(self.arrhenius_results.values()):
            exibir_erro("Nenhum resultado para exportar. Execute o cálculo primeiro.")
            return

        exibir_mensagem("Preparando exportação...")
        df_list = []
        for idx, data in self.arrhenius_results.items():
            if not data:
                continue
            df_temp = pd.DataFrame(data, columns=['DensidadePct', 'InvT', 'ln_Rate_div_T'])
            df_temp['Ensaio'] = idx + 1
            df_list.append(df_temp)

        if not df_list:
            with self.out:
                exibir_erro("Nenhum dado válido para exportar.")
            return

        df_export = pd.concat(df_list, ignore_index=True)
        link = gerar_link_download(df_export, nome_arquivo="Arrhenius_Results.xlsx")
        display(link)
        exibir_mensagem("Exportação de Arrhenius concluída.")

    def get_results(self):
        return self.arrhenius_results


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from collections import defaultdict

# Utilitários do Módulo 1
from modulo1_interface import exibir_mensagem, exibir_erro, gerar_link_download, R

class Modulo6_1ArrheniusDisplay:
    """
    Submódulo para Exibição de Curvas Arrhenius e Tabela de Ea

    1) Recebe arrhenius_results (dict) do passo anterior:
       arrhenius_results[i] = [(dens, inv_T, lnVal), ...]
    2) Combina os pontos de todos os ensaios e agrupa por densidade.
    3) Faz regressão linear ln(...) vs. 1/T para cada densidade.
    4) Exibe gráfico único com todas as densidades e retas,
       e gera tabela 'dens vs Ea'.
    5) Exporta em Excel e possui botão Back.
    """

    def __init__(self, arrhenius_results: dict, n_ensaios: int, parent=None):
        self.arrhenius_results = arrhenius_results
        self.n_ensaios = n_ensaios
        self.parent = parent
        self.results_df = None

        self.out = widgets.Output()
        self.btn_plot = widgets.Button(description="Plotar & Mostrar Tabela", button_style='success')
        self.btn_export = widgets.Button(description="Exportar Tabela de Ea", button_style='info', disabled=True)
        self.btn_back = widgets.Button(description="Voltar", button_style='warning')

        self.btn_plot.on_click(self._on_plot)
        self.btn_export.on_click(self._on_export)
        self.btn_back.on_click(self._on_back_clicked)

        self._build_ui()

    def _build_ui(self):
        if self.n_ensaios < 2:
            self.main_ui = widgets.HTML(
                "<b>O método de Arrhenius requer pelo menos 2 ensaios com taxas de aquecimento distintas.</b>"
            )
        else:
            explanation = (
                "<div style='font-family: Arial, sans-serif; line-height: 1.5;'>"
                "<h3>Visualização da Análise de Arrhenius</h3>"
                "<p>Combina pontos (ln(taxa/T) vs. 1/T) de todos os ensaios, agrupando por densidade.</p>"
                "<p>Cada grupo passa por regressão linear: Ea = -slope × R.</p>"
                "</div>"
            )
            self.main_ui = widgets.VBox([
                widgets.HTML(value=explanation),
                widgets.HBox([self.btn_plot, self.btn_export, self.btn_back]),
                self.out
            ])

    def display(self):
        clear_output(wait=True)
        display(self.main_ui)

    def _on_plot(self, _):
        with self.out:
            clear_output()

            if not self.arrhenius_results:
                exibir_erro("[ERRO] Nenhum resultado de Arrhenius encontrado. Execute o cálculo no módulo anterior.")
                return

            dens_map = defaultdict(list)
            for ensaio, arr_list in self.arrhenius_results.items():
                for dens, invT, lnVal in arr_list:
                    if not np.isfinite(dens) or not np.isfinite(invT) or not np.isfinite(lnVal):
                        continue
                    dens_map[round(dens, 2)].append((invT, lnVal))

            if not dens_map:
                exibir_erro("[ERRO] Não há dados válidos para plotar.")
                return

            fig, ax = plt.subplots(figsize=(8, 6))
            ax.set_title("Arrhenius por Nível de Densidade")
            ax.set_xlabel("1/T (K⁻¹)")
            ax.set_ylabel("ln[(dρ/dt)/T]")

            results = []
            for dens, points in sorted(dens_map.items()):
                if len(points) < 2:
                    continue
                x_vals = np.array([pt[0] for pt in points])
                y_vals = np.array([pt[1] for pt in points])

                try:
                    slope, intercept, r_val, *_ = linregress(x_vals, y_vals)
                    if r_val**2 < 0.8:
                        continue
                    Ea_calc = -slope * R / 1000  # converte para kJ/mol
                except Exception as e:
                    exibir_erro(f"[ERRO] Regressão falhou para densidade {dens}%: {e}")
                    continue

                ax.plot(x_vals, y_vals, 'o', label=f"ρ={dens}% (R²={r_val**2:.2f})")
                fit_x = np.array([x_vals.min(), x_vals.max()])
                ax.plot(fit_x, slope * fit_x + intercept, '-', label=None)
                results.append((dens, Ea_calc, r_val**2))

            if not results:
                exibir_erro("[ERRO] Pontos insuficientes para regressão confiável.")
                return

            ax.legend(bbox_to_anchor=(1.05,1), loc='upper left')
            ax.grid(True)
            plt.tight_layout()
            display(fig)

            # Monta DataFrame de resultados
            self.results_df = pd.DataFrame(results, columns=["Densidade(%)", "Ea (kJ/mol)", "R_quadrado"])
            self.btn_export.disabled = False

            display(HTML(self.results_df.to_html(index=False, float_format="{:.2f}".format)))

    def _on_export(self, _):
        with self.out:
            clear_output()
            if self.results_df is None or self.results_df.empty:
                exibir_erro("[ERRO] Nenhum resultado para exportar.")
                return
            exibir_mensagem("[INFO] Exportando tabela de Ea...")
            link = gerar_link_download(self.results_df, nome_arquivo="Arrhenius_Ea_Table.xlsx")
            display(link)
            exibir_mensagem("[INFO] Exportação concluída.")

    def _on_back_clicked(self, _):
        with self.out:
            clear_output()
            exibir_mensagem("[INFO] Retornando...")
        # Navega conforme controlador
        if self.parent and hasattr(self.parent, 'show_module_by_step'):
            self.parent.show_module_by_step(self.parent.STEP_MAP.get('METHOD_CHOICE'))
        elif self.parent and hasattr(self.parent, 'display'):
            self.parent.display()
        else:
            exibir_erro("[ERRO] Método de retorno não definido.")


In [ ]:
import copy
import datetime

class DataHistory:
    """
    Classe para armazenar o histórico dos DataFrames.

    Cada registro armazena:
      - A cópia profunda do DataFrame (para evitar alterações futuras)
      - O nome do módulo de onde os dados vieram
      - Os nomes das colunas no momento do armazenamento
      - Um timestamp indicando quando o estado foi salvo.
    """
    def __init__(self):
        self.history = []  # Lista para armazenar os registros de histórico

    def push(self, data, module_name: str):
        """
        Salva uma cópia do DataFrame 'data', juntamente com informações do módulo.

        Args:
            data (pd.DataFrame): O DataFrame a ser armazenar.
            module_name (str): Identificador do módulo de origem.
        """
        timestamp = datetime.datetime.now()
        record = {
            'timestamp': timestamp,
            'module': module_name,
            'columns': list(data.columns),
            'data': copy.deepcopy(data)
        }
        self.history.append(record)
        # Log amigável para o usuário
        print(f"[HISTORY] [{timestamp:%Y-%m-%d %H:%M:%S}] Salvado: '{module_name}' com colunas {record['columns']}")

    def pop(self):
        """
        Remove e retorna o registro mais recente do histórico.

        Returns:
            dict | None: O registro removido ou None se histórico vazio.
        """
        if not self.history:
            print("[HISTORY] Histórico vazio. Nada a restaurar.")
            return None
        record = self.history.pop()
        ts = record['timestamp']
        print(f"[HISTORY] Removido: '{record['module']}' salvo em {ts:%Y-%m-%d %H:%M:%S}")
        return record

    def peek(self):
        """
        Retorna o registro mais recente sem removê-lo.

        Returns:
            dict | None: O registro mais recente ou None se histórico vazio.
        """
        if not self.history:
            return None
        return self.history[-1]

    def get_all(self):
        """
        Retorna todos os registros de histórico.

        Returns:
            list: Lista de registros de histórico.
        """
        return list(self.history)

In [ ]:
# ==============================================================================
# CONTROLADOR PRINCIPAL (MAIN INTERACTIVE) - VERSÃO HÍBRIDA
# ==============================================================================
import ipywidgets as widgets
from IPython.display import display, clear_output

# Import de todos os seus módulos de UI e processamento
from modulo2_importacao import Modulo2Importacao
from modulo3_filtrorecorte import Modulo3Recorte
# Import do MÓDULO NOVO que criamos
from modulo4_simulacao_hibrida import Modulo4SimulacaoHibrida
# O nome original dos seus módulos seguintes foi mantido para compatibilidade
from modulo4_logtheta import ModuloLogTheta
from modulo5_1_alinhamento import Modulo5_1Alinhamento
from modulo5_2_blaine import Modulo5_2BlaineParameters
from modulo5_3_sigmoides import Modulo5_3Sigmoides
from modulo5_3_1_revisao import Modulo5_3_1_Revisao
from modulo5_4_roteador import Modulo5_4_Roteador
from modulo5_5_refinamento import Modulo5_5_Refinamento
from modulo6_0_arrhenius import Modulo6_0_Arrhenius
from modulo6_1_arrhenius_display import Modulo6_1ArrheniusDisplay

# Utilitários de interface do seu arquivo core
from core import exibir_mensagem, exibir_erro

class MainInteractive:
    """
    Controlador principal refatorado para integrar o fluxo de simulação híbrida.
    Gerencia a navegação entre os módulos e o estado da aplicação.
    """
    def __init__(self):
        # Armazenamento de estado
        self.sintering_records = []
        self.analysis_choice = 'logtheta' # Padrão
        self.current_step_name = 'INTRO'

        # Instâncias de Módulos
        self.modules = {}

        # Sequência de passos para o caminho mais longo (LogTheta)
        self.step_sequence = [
            'INTRO',
            'IMPORT',
            'FILTER',
            'METHOD_CHOICE',
            'LOGTHETA_ROUTER', # Roteador para escolher entre análise real ou híbrida
            'LOGTHETA_CALC',   # Cálculo do LogTheta
            'ALIGN',
            'BLAINE',
            'FIT_SELECTION',
            'REVIEW',
            'FINAL_ANALYSIS',
            'REFINEMENT'
        ]

        self.arrhenius_sequence = ['INTRO', 'IMPORT', 'FILTER', 'METHOD_CHOICE', 'ARRHENIUS_CALC', 'ARRHENIUS_DISPLAY']

        # Widgets da UI
        self._build_ui()
        self.show_module_by_step('INTRO')

    def _build_ui(self):
        self.step_label = widgets.HTML()
        self.nav_buttons = {
            'next': widgets.Button(description="Próxima Etapa >>", button_style='success'),
            'back': widgets.Button(description="<< Etapa Anterior", button_style='warning')
        }
        self.nav_buttons['next'].on_click(self._on_next_step)
        self.nav_buttons['back'].on_click(self._on_back_step)

        self.log_area = widgets.Output()
        self.modules_container = widgets.VBox([])
        nav_bar = widgets.HBox([self.nav_buttons['back'], self.nav_buttons['next']])
        self.main_ui = widgets.VBox([nav_bar, self.step_label, self.log_area, self.modules_container])

    def display(self):
        display(self.main_ui)

    def _update_nav_buttons(self):
        # Desabilita o botão 'Voltar' na primeira etapa
        is_first_step = (self.step_sequence.index(self.current_step_name) <= 1 if self.analysis_choice == 'logtheta' else self.arrhenius_sequence.index(self.current_step_name) <= 1)
        self.nav_buttons['back'].disabled = is_first_step

        # Desabilita o botão 'Próximo' em etapas com navegação interna
        interactive_steps = ['METHOD_CHOICE', 'REVIEW', 'FINAL_ANALYSIS', 'REFINEMENT', 'ARRHENIUS_DISPLAY', 'LOGTHETA_ROUTER']
        self.nav_buttons['next'].disabled = (self.current_step_name in interactive_steps)

    def _on_next_step(self, _=None):
        with self.log_area: clear_output(wait=True)

        # Validação da etapa atual antes de avançar
        if self.current_step_name == 'IMPORT':
            if not self.modules.get('mod2') or not self.modules['mod2'].validar_etapa():
                exibir_erro("Validação da Etapa 1 falhou. Verifique se todos os dados foram carregados, mapeados e convertidos.")
                return

        sequence = self.step_sequence if self.analysis_choice == 'logtheta' else self.arrhenius_sequence
        current_index = sequence.index(self.current_step_name)
        if current_index < len(sequence) - 1:
            self.show_module_by_step(sequence[current_index + 1])

    def _on_back_step(self, _=None):
        with self.log_area: clear_output(wait=True)
        sequence = self.step_sequence if self.analysis_choice == 'logtheta' else self.arrhenius_sequence
        current_index = sequence.index(self.current_step_name)
        if current_index > 0:
            self.show_module_by_step(sequence[current_index - 1])

    def show_module_by_step(self, step_name: str):
        self.current_step_name = step_name
        self.modules_container.children = ()

        # Mapeamento de nomes de passos para funções de construção
        step_runners = {
            'INTRO': self._show_intro, 'IMPORT': self._show_import, 'FILTER': self._show_filter,
            'METHOD_CHOICE': self._show_method_choice,
            'LOGTHETA_ROUTER': self._show_logtheta_router,
            'LOGTHETA_CALC': self._show_logtheta_calc,
            'ARRHENIUS_CALC': self._show_arrhenius_calc,
            'ARRHENIUS_DISPLAY': self._show_arrhenius_display,
            'ALIGN': self._show_align, 'BLAINE': self._show_blaine, 'FIT_SELECTION': self._show_fit_selection,
            'REVIEW': self._run_fit_and_show_review, 'FINAL_ANALYSIS': self._show_final_analysis_router,
            'REFINEMENT': self.show_refinement_module
        }

        # Executa a função correspondente ao passo
        action = step_runners.get(step_name)
        if action:
            action()
        self._update_nav_buttons()

    # --- Funções de construção para cada passo ---

    def _show_intro(self):
        self.step_label.value = "<h3>Passo 0: Introdução</h3>"
        self.modules_container.children = (widgets.HTML("<h1>Bem-vindo ao Ogum Sintering</h1><p>Clique em 'Próxima Etapa' para começar a importação de dados.</p>"),)

    def _show_import(self):
        self.step_label.value = "<h3>Passo 1: Importar e Mapear Dados</h3>"
        if 'mod2' not in self.modules: self.modules['mod2'] = Modulo2Importacao()
        self.modules_container.children = (self.modules['mod2'].main_ui,)

    def _show_filter(self):
        self.step_label.value = "<h3>Passo 2: Recortar e Filtrar Dados</h3>"
        # Pega a saída do módulo anterior
        dfs_from_mod2 = self.modules['mod2'].get_dfs()
        # Instancia o módulo de filtro
        self.modules['mod3'] = Modulo3Recorte(dfs_from_mod2)
        self.modules_container.children = (self.modules['mod3'].main_ui,)

    def _show_method_choice(self):
        self.step_label.value = "<h3>Passo 3: Escolher Método de Análise</h3>"
        radio = widgets.RadioButtons(options=[("Curva Mestra (LogTheta)", "logtheta"), ("Arrhenius", "arrhenius")], description="Método:")
        btn = widgets.Button(description="Confirmar e Continuar", button_style='info')
        def on_confirm(_):
            self.analysis_choice = radio.value
            # Validação específica do método
            if self.analysis_choice == 'arrhenius' and len(self.modules['mod3'].get_dfs()) < 2:
                with self.log_area:
                    clear_output()
                    exibir_erro("O método de Arrhenius requer pelo menos 2 ensaios experimentais com taxas de aquecimento distintas.")
                return

            # Navega para o próximo passo apropriado
            next_step = 'LOGTHETA_ROUTER' if self.analysis_choice == 'logtheta' else 'ARRHENIUS_CALC'
            self.show_module_by_step(next_step)
        btn.on_click(on_confirm)
        self.modules_container.children = (widgets.VBox([widgets.HTML("<h4>Escolha o método de análise principal:</h4>"), radio, btn]),)

    def _show_logtheta_router(self):
        self.step_label.value = "<h3>Passo 4: Preparar Dados para Curva Mestra</h3>"
        dfs_from_mod3 = self.modules['mod3'].get_dfs()
        # Instancia o nosso novo módulo roteador
        self.modules['router'] = Modulo4SimulacaoHibrida(dfs_from_mod3)
        self.modules_container.children = (self.modules['router'].main_ui,)
        # Desabilita o 'Próximo' pois a navegação é interna
        self.nav_buttons['next'].disabled = True

    def _show_logtheta_calc(self):
        self.step_label.value = "<h3>Passo 5: Calcular LogTheta</h3>"
        # Pega o DataFrame de saída do roteador
        df_concatenado = self.modules['router'].get_final_dataframe()
        if df_concatenado is None or df_concatenado.empty:
            exibir_erro("O módulo anterior não produziu dados para análise. Volte e execute o passo anterior.")
            return

        # CRUCIAL: O Módulo de LogTheta espera uma LISTA de DataFrames.
        # Nós precisamos re-dividir o DataFrame concatenado.
        dfs_para_logtheta = [group.drop('Ensaio', axis=1) for _, group in df_concatenado.groupby('Ensaio')]

        self.modules['logtheta'] = ModuloLogTheta(dfs_para_logtheta)
        self.modules_container.children = (self.modules['logtheta'].ui,)

    def _show_arrhenius_calc(self):
        self.step_label.value = "<h3>Passo 4 (Arrhenius): Calcular Pontos</h3>"
        dfs_from_mod3 = self.modules['mod3'].get_dfs()
        self.modules['arrhenius_calc'] = Modulo6_0_Arrhenius(dfs_from_mod3)
        self.modules_container.children = (self.modules['arrhenius_calc'].main_ui,)

    def _show_arrhenius_display(self):
        self.step_label.value = "<h3>Passo 5 (Arrhenius): Análise e Resultados</h3>"
        arrhenius_results = self.modules['arrhenius_calc'].get_results()
        n_ensaios = len(self.modules['mod2'].get_dfs())
        self.modules['arrhenius_disp'] = Modulo6_1ArrheniusDisplay(arrhenius_results, n_ensaios, parent=self)
        self.modules_container.children = (self.modules['arrhenius_disp'].main_ui,)

    # O restante das funções de exibição (align, blaine, etc.) permanecem as mesmas
    # pois a lógica de pegar a saída do módulo anterior e passar para o próximo é mantida

    def _show_align(self):
        self.step_label.value = "<h3>Passo 6: Alinhamento e Interpolação</h3>"
        self.sintering_records = self.modules['logtheta'].get_sintering_records()
        if not self.sintering_records:
             exibir_erro("Nenhum dado de LogTheta foi calculado. Volte e execute o passo anterior.")
             return
        self.modules['align'] = Modulo5_1Alinhamento(self.sintering_records)
        self.modules_container.children = (self.modules['align'].main_ui,)

    def _show_blaine(self):
        self.step_label.value = "<h3>Passo 7: Calcular Parâmetros de Blaine</h3>"
        self.sintering_records = self.modules['align'].get_aligned_records()
        self.modules['blaine'] = Modulo5_2BlaineParameters(self.sintering_records)
        self.modules_container.children = (self.modules['blaine'].main_ui,)

    def _show_fit_selection(self):
        self.step_label.value = "<h3>Passo 8: Selecionar Dados para Ajuste Sigmoidal</h3>"
        self.sintering_records = self.modules['blaine'].get_blaine_records()
        self.modules['sigmoidal'] = Modulo5_3Sigmoides(self.sintering_records)
        self.modules_container.children = (self.modules['sigmoidal'].main_ui,)

    def _run_fit_and_show_review(self, p0_overrides=None):
        self.step_label.value = "<h3>Passo 9: Revisar e Validar Ajustes</h3>"
        mod_sigmoidal = self.modules['sigmoidal']
        if not p0_overrides:
            ui_selection = mod_sigmoidal.select_data.value
            if not ui_selection:
                exibir_erro("Nenhum tipo de dado foi selecionado para o ajuste."); self.show_module_by_step('FIT_SELECTION'); return
            types_to_fit = [mod_sigmoidal.DATA_OPTIONS_MAP[ui_name] for ui_name in ui_selection]
        else:
            types_to_fit = list(set(key[2] for key in p0_overrides.keys()))
        with self.log_area: clear_output(wait=True)
        mod_sigmoidal.run_all_fits(types_to_fit, p0_overrides)
        self.sintering_records = mod_sigmoidal.get_all_records_after_fit()
        self.modules['review'] = Modulo5_3_1_Revisao(self.sintering_records, controller=self)
        self.modules_container.children = (self.modules['review'].main_ui,)

    def _show_final_analysis_router(self):
        self.step_label.value = "<h3>Passo 10: Análise Final e Busca da Melhor Ea</h3>"
        self.modules['router5_4'] = Modulo5_4_Roteador(self.sintering_records, controller=self)
        self.modules_container.children = (self.modules['router5_4'].main_ui,)

    def show_refinement_module(self, initial_ea, target_curve_type):
        self.step_label.value = "<h3>Passo 11: Refinamento da Energia de Ativação</h3>"
        self.current_step_name = 'REFINEMENT'
        self.modules['refinement'] = Modulo5_5_Refinamento(
            sintering_records=self.sintering_records,
            initial_ea=initial_ea,
            target_curve_type=target_curve_type,
            parent=self
        )
        self.modules_container.children = (self.modules['refinement'].main_ui,)
        self._update_nav_buttons()

    def trigger_refit(self, p0_overrides):
        self._run_fit_and_show_review(p0_overrides=p0_overrides)

# --- Para executar a aplicação ---#
main_app = MainInteractive()
main_app.display()